In [2]:
from bs4 import BeautifulSoup     #网页解析，获取数据\n",
import re       #正则表达式，进行文字匹配\n",
import urllib.request,urllib.error      #制定URL，获取网页数据\n",
# import xlwt     #进行excel操作\n",
import sqlite3  #进行SQLite数据库操作\n",
import pandas as pd #存储数据及处理数据\n",
import numpy as np
import requests
import random
import time
from collections import defaultdict
from collections import deque
from collections import Counter
import collections
import json
import sys
import datetime
import os
import csv
from math import *
import pymysql
# from geopy.distance import geodesic
# from pyproj import Proj
pd.set_option("display.max_columns", 100)

from scipy.interpolate import interp1d

###

### 基于企业数量的产业图谱绘制

In [7]:
# 2024年全量企业数据读取。
df_chrunks=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\t_enterprise_all_2024.csv',dtype='str',chunksize=200000)
# 地区编码
df_district_code=pd.read_csv(r'\\172.30.10.237\ict\二所（数智经济研究所）\生产技术委员会\公开数据\2023082110民政部地区编码\admin_division.csv',dtype='str')
# 行业编码
df_ind_code=pd.read_csv(r'E:\project_basedata\2023082101国标行业\national_industry_17.csv',dtype='str')
df_ind_code_1=df_ind_code.drop_duplicates(subset=['Id_total','Label'],keep='first')

In [8]:
# 企业大文件处理
df_result=pd.DataFrame()
# count=1
for chrunk in df_chrunks:
    
    chrunk.loc[:,'location']=chrunk['credit_no'].str[2:8]
    df_i=chrunk.groupby(['location','industry_code'])['credit_no'].count().reset_index()
    df_result=pd.concat([df_result,df_i],axis=0)
    
    # print(count)
    # count+=1
    # print(df_i)
    # sys.exit()
df_result_i=df_result.groupby(['location','industry_code'])['credit_no'].sum().reset_index()
df_result_i_1=pd.merge(left=df_result_i,right=df_district_code,left_on=['location'],right_on=['type_4'],how='left')
df_result_i_2=df_result_i_1.loc[df_result_i_1['type_4'].notnull()]
df_result_i_3=pd.merge(left=df_result_i_2,right=df_ind_code_1,left_on=['industry_code'],right_on=['Id_total'],how='left')
df_result_i_4=df_result_i_3.loc[df_result_i_3['Id_total'].notnull()]

        location industry_code  credit_no  type_4 admin_name    type_x     Id   
0         100000         A0100          1  100000         中国  district    A01  \
1         100000         A0190          1  100000         中国  district   A019   
2         100000         A0241          1  100000         中国  district  A0241   
3         100000         A0530          1  100000         中国  district   A053   
4         100000         B0700          1  100000         中国  district    B07   
...          ...           ...        ...     ...        ...       ...    ...   
1892450   659402         R9013          1  659402       一零一团  district  R9013   
1892451   712223         F5149          1  712223        头城镇  district  F5149   
1892452   712301         F5139          1  712301        桃园市  district  F5139   
1892453   712825         F5129          1  712825        名间乡  district  F5129   
1892454   810306         S9500          1  810306        西贡区  district    S95   

        Id_total           

In [ ]:
# 保存2024年企业的地区行业数量数据
df_result_i_4.to_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\t_enterprise_all_2024_city_ind_num.csv',index=False)

### 计算2024各城市各行业小类rca

In [9]:
df_result_i_4.loc[:,'city']=df_result_i_4['location'].str[:4]

# 把直辖市的数据统一 1100和1101 3100和3101 5000和5001 1200和1201
df_result_i_4.loc[(df_result_i_4['city'].notnull())&(df_result_i_4['city'].str.startswith('11')),'city']='1101'
df_result_i_4.loc[(df_result_i_4['city'].notnull())&(df_result_i_4['city'].str.startswith('12')),'city']='1201'
df_result_i_4.loc[(df_result_i_4['city'].notnull())&(df_result_i_4['city'].str.startswith('31')),'city']='3101'
df_result_i_4.loc[(df_result_i_4['city'].notnull())&(df_result_i_4['city'].str.startswith('50')),'city']='5001'
# 筛选城市
df_result_i_5=df_result_i_4.groupby(['city','industry_code'])['credit_no'].sum().reset_index()
df_result_i_6=df_result_i_5.loc[df_result_i_5['city']!='1000']
# 筛选行业小类
ls_ind_subclass=df_ind_code.loc[df_ind_code['type']=='subclass']['Id'].unique().tolist()
df_result_i_7=df_result_i_6.loc[df_result_i_6['industry_code'].isin(ls_ind_subclass)]

# 算rca
df_result_i_7_city_num=df_result_i_7.groupby(['city'])['credit_no'].sum().reset_index()
df_result_i_7_city_num.rename(columns={'credit_no':'city_num'},inplace=True)
df_result_i_7_ind_num=df_result_i_7.groupby(['industry_code'])['credit_no'].sum().reset_index()
df_result_i_7_ind_num.rename(columns={'credit_no':'ind_num'},inplace=True)

df_result_i_8=pd.merge(left=df_result_i_7,right=df_result_i_7_city_num,on=['city'],how='left')
df_result_i_9=pd.merge(left=df_result_i_8,right=df_result_i_7_ind_num,on=['industry_code'],how='left')
total_num=df_result_i_9['credit_no'].sum()
df_result_i_9.loc[:,'total_num']=total_num
df_result_i_9.loc[:,'rca']=(df_result_i_9['credit_no']/df_result_i_9['city_num'])/(df_result_i_9['ind_num']/df_result_i_9['total_num'])


        city industry_code  credit_no  city_num  ind_num  total_num   
0       1101         A0111        293   1871320   248715   44786850  \
1       1101         A0112         64   1871320   145295   44786850   
2       1101         A0113        116   1871320    88458   44786850   
3       1101         A0119        682   1871320   130666   44786850   
4       1101         A0121         38   1871320    17737   44786850   
...      ...           ...        ...       ...      ...        ...   
318616  6594         P8393          2       102    92507   44786850   
318617  6594         R9013          1       102    33577   44786850   
318618  7122         F5149          1         1    39755   44786850   
318619  7123         F5139          1         1   134189   44786850   
318620  7128         F5129          1         1   274574   44786850   

                rca  
0          0.028195  
1          0.010542  
2          0.031385  
3          0.124918  
4          0.051275  
...            

In [38]:
# 保存rca计算结果
df_result_i_9.to_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\2024rca.csv',index=False)

In [ ]:
df_result_i_9=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\2024rca.csv')

In [10]:
df_result_i_9=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\2024rca.csv')
# 图谱绘制去掉城市企业数量末位10%的城市
city_num_quantile=df_result_i_7_city_num.drop_duplicates()['city_num'].quantile(0.2) # 19055.8
df_result_i_9_=df_result_i_9.loc[df_result_i_9['city_num']>=city_num_quantile]
# 平均rca值用以判断优势行业
df_result_i_9_1=df_result_i_9_.groupby(['industry_code'])['rca'].mean().reset_index()
df_result_i_9_1.rename(columns={'rca':'rca_mean'},inplace=True)
df_result_i_9_2=pd.merge(left=df_result_i_9_,right=df_result_i_9_1,on=['industry_code'])
# 计算共现概率
ls_ind=df_result_i_9_2['industry_code'].unique().tolist()
dq_ind=deque(ls_ind)

In [ ]:
# 构建全联通产业优势共现关系
start_time=time.time()
ls_result=[]
while len(dq_ind)>0:
    start=dq_ind.popleft()
    for j in dq_ind:
        df_start=df_result_i_9_2.loc[(df_result_i_9_2['industry_code']==start)&(df_result_i_9_2['rca']>1)&(df_result_i_9_2['rca']>df_result_i_9_2['rca_mean'])]
        n_start=len(df_start)
        df_end=df_result_i_9_2.loc[(df_result_i_9_2['industry_code']==j)&(df_result_i_9_2['rca']>1)&(df_result_i_9_2['rca']>df_result_i_9_2['rca_mean'])]
        n_end=len(df_end)
        df_cooccurance=pd.merge(left=df_start,right=df_end,on=['city'],how='inner')
        n_cooccurance=len(df_cooccurance)
        if n_start>n_end:
            weight=n_cooccurance/n_start
        else:
            weight=n_cooccurance/n_end
        ls_result.append([start,j,str(weight)])
print('time_cost:',str(time.time()-start_time))

       Source Target    Weight
0       A0111  A0112  0.151899
1       A0111  A0113  0.215190
2       A0111  A0119  0.367089
3       A0111  A0121  0.177215
4       A0111  A0122  0.443038
...       ...    ...       ...
935023  C3934  S9226  0.000000
935024  C3934  S9513  0.000000
935025  J6636  S9226  0.000000
935026  J6636  S9513  0.000000
935027  S9226  S9513  0.000000

[935028 rows x 3 columns]


In [7]:
pd.DataFrame(ls_result,columns=['Source','Target','Weight']).to_csv(r'E:\project_basedata\2024040301行业图谱\total_connect_.csv',index=False)

In [2]:
# 最小生成树
# 查找根节点
def query(nodes,x):
    if nodes[x]==x:
        return x
    return query(nodes,nodes[x])

# 新增节点合并
def union(nodes,start,end):
    nodes[query(nodes,end)]=query(nodes,start)

In [3]:
# 最小生成树修剪
df_connect_=pd.read_csv(r'E:\project_basedata\2024040301行业图谱\total_connect_.csv')
df_connect=df_connect_.loc[df_connect_['Weight']>0]

# 只筛选二产
df_connect.loc[:,'Source_type']=df_connect['Source'].str[0]
df_connect.loc[:,'Target_type']=df_connect['Target'].str[0]
df_2chan=df_connect.loc[df_connect['Source_type'].isin(['C','I','M'])&(df_connect['Target_type'].isin(['C','I','M']))]
df_2chan_=df_2chan.copy()

# 最小生成树
start = time.time()
mst = pd.DataFrame()
# 首先构造一个字典,每个点的值为该点的根节点
nodes=dict()
ls1=df_2chan_['Source'].unique().tolist()
ls2=df_2chan_['Target'].unique().tolist()
ls=ls1+ls2
ls=list(set(ls))
for i in ls:
    nodes[i]=i
    
# 根据联系数据，构建最小生成树
df_relation_sk_sort=df_2chan_.sort_values(by=['Weight'],ascending=False).reset_index(drop=True)
for i in range(len(df_relation_sk_sort)):
    start_i=df_relation_sk_sort.loc[i,'Source']
    end_i=df_relation_sk_sort.loc[i,'Target']
    if query(nodes,start_i) != query(nodes,end_i):
        union(nodes,start_i,end_i)
        mst=pd.concat([mst,df_relation_sk_sort.loc[[i]]])

end = time.time()
runTime = end - start
print("运行时间：", runTime)

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_19340\2865287815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_connect.loc[:,'Source_type']=df_connect['Source'].str[0]
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_19340\2865287815.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_connect.loc[:,'Target_type']=df_connect['Target'].str[0]


运行时间： 13.943783521652222


In [4]:
print(len(mst))
mst.to_csv(r'E:\project_basedata\2024040301行业图谱\graph_2024_2chan_删去农业和采矿.csv',index=False)

690


In [5]:
# 从全量联系中由大到小补充x条联系，达到节点数量3倍为最佳
mst_=mst.copy()
mst_.loc[:,'Id_no']='total'
df_relation_sk_sort=df_2chan_.sort_values(by=['Weight'],ascending=False).reset_index(drop=True)
df_filt=pd.merge(left=mst_,right=df_relation_sk_sort,on=['Source','Target','Weight'],how='right')
df_concat=df_filt.loc[df_filt['Id_no'].isnull(),['Source','Target','Weight']].head(1400)
mst_p=pd.concat([mst,df_concat],axis=0)

In [7]:
mst_p.to_csv(r'E:\project_basedata\2024040301行业图谱\2024_2chan\graph_2024_2chan_p_graph_删去农业和采矿.csv',index=False)

In [10]:
# gephi点文件生成
ls1=mst_p['Source'].unique().tolist()
ls2=mst_p['Target'].unique().tolist()
ls=list(set(ls1+ls2))
df_nodes=pd.DataFrame(ls,columns=['Id'])
df_nodes_=pd.merge(left=df_nodes,right=df_ind_code,on=['Id'],how='left')
df_nodes_.loc[:,'class']=df_nodes_['Id'].str[0]
df_nodes_.loc[:,'medium_class']=df_nodes_['Id'].str[0:4]

df_ind_code_=df_ind_code.loc[df_ind_code['type']=='class',['Id','Label']]
df_ind_code_.rename(columns={'Id':'class','Label':'type_Label'},inplace=True)
df_nodes_1=pd.merge(left=df_nodes_.loc[:,['Id','Label','class','medium_class']],right=df_ind_code_,on=['class'],how='left')

df_ind_code_1=df_ind_code.loc[df_ind_code['type']=='medium',['Id','Label']]
df_ind_code_1.rename(columns={'Id':'medium_class','Label':'medium_Label'},inplace=True)
df_nodes_2=pd.merge(left=df_nodes_1,right=df_ind_code_1,on=['medium_class'],how='left')
df_nodes_2.to_csv(r'E:\project_basedata\2024040301行业图谱\2024_2chan\graph_2024_2chan_nodes_删去农业和采矿.csv',index=False)

In [43]:
df_ind_code_1

,medium_class,medium_Label,type
117,A011,谷物种植,medium
118,A012,豆类、油料和薯类种植,medium
119,A013,棉、麻、糖、烟草种植,medium
120,A014,蔬菜、食用菌及园艺作物种植,medium
121,A015,水果种植,medium
...,...,...,...
585,S953,基金会,medium
586,S954,宗教组织,medium
587,S961,社区居民自治组织,medium
588,S962,村民自治组织,medium


In [45]:
df_nodes_2

,Id,Label,class,medium_class,type_Label,medium_Label
0,C3219,其他常用有色金属冶炼,C,C321,制造业,常用有色金属冶炼
1,C3553,缝制机械制造,C,C355,制造业,纺织、服装和皮革加工专用设备制造
2,C1942,羽毛(绒)制品加工,C,C194,制造业,羽毛(绒)加工及制品制造
3,I6432,互联网生活服务平台,I,I643,信息传输、软件和信息技术服务业,互联网平台
4,C4040,光学仪器制造,C,C404,制造业,光学仪器制造
...,...,...,...,...,...,...
797,A0151,仁果类和核果类水果种植,A,A015,农、林、牧、渔业,水果种植
798,C3056,玻璃保温容器制造,C,C305,制造业,玻璃制品制造
799,C1312,小麦加工,C,C131,制造业,谷物磨制
800,B0933,放射性金属矿采选,B,B093,采矿业,稀有稀土金属矿采选


###

### 产业图谱模拟

In [69]:
def simulate(code_ls,df_net_new,threshold):
    _district_industryCOs=code_ls
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast = []
    while flag == 1:
        _exist_list = []
        for k in range(0,len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []
        for j in range(len(_simulate_list[len(_simulate_list)-1])):
            _industryCO = _simulate_list[len(_simulate_list)-1][j]
            #if _industryCO in industryCOs_product:
            # 从每一个优势产业开始向外找，一步联系值超过的节点
            _df_industryCO_proximity = df_net_new.loc[((df_net_new['Source']==_industryCO) | (df_net_new['Target']==_industryCO))&(df_net_new['Weight']>=threshold)]
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('上游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_a.append('下游')
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1
        if c == 0:
            flag=0
        else:
            flag = 1
            t = t + 1
            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

In [2]:
df_total = pd.read_csv(r'E:\project_tupu\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_ind = pd.read_excel(r'E:\project_basedata\industry_new.xlsx',dtype='str')
df_ind=df_ind.rename(columns={'小类代码':'Id'}).copy()
ls_ind_4=df_ind['小类类别代码'].unique().tolist()
df_dis_2=pd.read_excel(r'E:\project_basedata\t_admin_division_code.xlsx',dtype='str')
ls_df_dis_2=df_dis_2['type_code'].unique().tolist()
df_total_filt=df_total.loc[(df_total['district_code'].isin(ls_df_dis_2))&(df_total['industry_code'].isin(ls_ind_4))]

In [2]:
df_total = pd.read_csv(r'E:\project_tupu\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})

# 城市的地区编码读取
df_city_code=pd.read_excel(r'E:\project_basedata\2023082110地区编码\t_admin_division_code_1125.xlsx',dtype='str',sheet_name='t_admin_division_code_city')
df_city_code=df_city_code.drop_duplicates(['type_4'],keep='first')
ls_city_code=df_city_code['type_4'].unique().tolist()
# 中类和小类行业读取
df_ind=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='industry_new')
df_ind.loc[:,'ind3']=df_ind['中类代码'].str[:4]
ls_ind4=df_ind['小类类别代码'].unique().tolist()
ls_ind3=df_ind['ind3'].unique().tolist()
df_medium=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='medium')
df_small=pd.read_excel(r'E:\project_basedata\2023082101国标行业\industry_new.xlsx',dtype='str',sheet_name='small')

In [3]:
# 图谱绘制
df_connect=pd.read_csv(r'E:\project_tupu\ind_connect_0705.csv',dtype='str')
df_connect=df_connect.astype(dtype={'Weight':'float'})
threshold = df_connect['Weight'].quantile(0.2)

### 结束

In [2]:

df_result_i_9=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\2024rca.csv')
df_result_i_9.loc[df_result_i_9['city']]

,city,industry_code,credit_no,city_num,ind_num,total_num,rca
0,1101,A0111,293,1871320,248693,44768490,0.028186
1,1101,A0112,64,1871320,145294,44768490,0.010538
2,1101,A0113,116,1871320,88422,44768490,0.031385
3,1101,A0119,682,1871320,130360,44768490,0.125160
4,1101,A0121,38,1871320,17730,44768490,0.051274
...,...,...,...,...,...,...,...
318581,6594,P8393,2,102,92494,44768490,9.490492
318582,6594,R9013,1,102,33571,44768490,13.073985
318583,7122,F5149,1,1,39740,44768490,1126.534726
318584,7123,F5139,1,1,134174,44768490,333.659949


In [ ]:
'91440101717852200L'

In [63]:
df_ns_gz.loc[df_ns_gz['行业代码'].str.startswith('C36')]

,地区编码,行业代码,num,num_dis,num_total_ind,total_num,rca,type_4,admin_name,short_name,short_name1,Id,Label,Id.1,ind_g
6976,440115,C3630,5,124977,796,38783772,1.949292,4401,南沙区,南沙,广州市,C3630,改装汽车制造,C3630,3630


In [34]:
# 广州南沙优势产业
# 广州优势产业
df_total4_1=pd.read_csv(r'E:\project_data\22nansha\tupu\gz_all_dis_rca.csv')
df_ns_gz=df_total4_1.loc[(df_total4_1['rca']>=1.3)&(df_total4_1['地区编码']==440115)]
df_total4_2=pd.read_excel(r'E:\project_basedata\2023090601启信宝企业统计数据\2022rca.xlsx')
df_gz=df_total4_2.loc[(df_total4_2['rca']>=1.6)&(df_total4_2['area_city']==4401)]
df_ns_gz.loc[:,'ind_g']=df_ns_gz['行业代码'].str[1:]
df_gz.loc[:,'ind_g']=df_gz['行业代码'].str[1:]

df_ns_gz=df_ns_gz.astype(dtype={'ind_g':'int'})
df_ns_gz=df_ns_gz.astype(dtype={'ind_g':'str'})

df_gz=df_gz.astype(dtype={'ind_g':'int'})
df_gz=df_gz.astype(dtype={'ind_g':'str'})
df_ns_gz=df_ns_gz.astype(dtype={'ind_g':'int'})
df_ns_gz=df_ns_gz.astype(dtype={'ind_g':'str'})

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_24680\1350963277.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ns_gz.loc[:,'ind_g']=df_ns_gz['行业代码'].str[1:]
C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_24680\1350963277.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gz.loc[:,'ind_g']=df_gz['行业代码'].str[1:]


In [56]:
code_ls_ns=df_ns_gz['ind_g'].unique().tolist()
code_ls_gz=df_gz['ind_g'].unique().tolist()

ls_result_ns=simulate(code_ls_ns,df_connect,threshold)
ls_result_gz=simulate(code_ls_gz,df_connect,threshold)
df_ns_result=pd.DataFrame(ls_result_ns,columns=['t','origin','destination','direction'])
df_gz_result=pd.DataFrame(ls_result_gz,columns=['t','origin','destination','direction'])
df_forcast_ns = df_ns_result.loc[df_ns_result['t'] <= 2]
df_forcast_gz = df_gz_result.loc[df_gz_result['t'] <= 2]
df_ns_result.to_csv(r'E:\project_data\22nansha\tupu\ns_for.csv',index=False)
df_gz_result.to_csv(r'E:\project_data\22nansha\tupu\gz_for.csv',index=False)

In [57]:
df_nodes=pd.read_csv(r'E:\project_data\18changsha\nodes.csv',dtype='str')
df_nodes_1=df_nodes.loc[:,['Id','Label','type1']]
df_nodes_3=df_nodes_1.copy()
df_nodes_3.loc[df_nodes_3['Id'].isin(code_ls_ns),'type2']='local_best'
df_nodes_3.loc[df_nodes_3['type2'].isnull(),'type2']='none'
df_nodes_4=pd.merge(left=df_nodes_3,right=df_ns_gz.loc[:,['ind_g','rca']],left_on=['Id'],right_on=['ind_g'],how='left')
df_nodes_4.loc[df_nodes_4['rca'].isnull(),'rca']=0
ori=df_forcast_ns.loc[(df_forcast_ns['t']==1)]['origin'].unique().tolist()
step_1=df_forcast_ns.loc[(df_forcast_ns['t']==1)]['destination'].unique().tolist()
step_2=df_forcast_ns.loc[(df_forcast_ns['t']==2)]['destination'].unique().tolist()
df_nodes_4.loc[df_nodes_4['Id'].isin(ori),'type3']='local_best'
df_nodes_4.loc[df_nodes_4['Id'].isin(step_1),'type3']='for1'
df_nodes_4.loc[df_nodes_4['Id'].isin(step_2),'type3']='for2'
df_nodes_4.loc[df_nodes_4['type3'].isnull(),'type3']='none'
df_nodes_4.to_csv(r'E:\project_data\22nansha\tupu\ns_nodes.csv',index=False)


df_nodes_gz=df_nodes_1.copy()
df_nodes_gz.loc[df_nodes_gz['Id'].isin(code_ls_gz),'type2']='local_best'
df_nodes_gz.loc[df_nodes_gz['type2'].isnull(),'type2']='none'
df_nodes_gz1=pd.merge(left=df_nodes_gz,right=df_gz.loc[:,['ind_g','rca']],left_on=['Id'],right_on=['ind_g'],how='left')
df_nodes_gz1.loc[df_nodes_gz1['rca'].isnull(),'rca']=0
ori=df_forcast_gz.loc[(df_forcast_gz['t']==1)]['origin'].unique().tolist()
step_1=df_forcast_gz.loc[(df_forcast_gz['t']==1)]['destination'].unique().tolist()
step_2=df_forcast_gz.loc[(df_forcast_gz['t']==2)]['destination'].unique().tolist()
df_nodes_gz1.loc[df_nodes_gz1['Id'].isin(ori),'type3']='local_best'
df_nodes_gz1.loc[df_nodes_gz1['Id'].isin(step_1),'type3']='for1'
df_nodes_gz1.loc[df_nodes_gz1['Id'].isin(step_2),'type3']='for2'
df_nodes_gz1.loc[df_nodes_gz1['type3'].isnull(),'type3']='none'
df_nodes_gz1.to_csv(r'E:\project_data\22nansha\tupu\gz_nodes.csv',index=False)
# 
# df_nodes_2=df_nodes_1.copy()
# df_nodes_2.loc[df_nodes_2['Id'].isin(ls_local_best),'type3']='local_best'
# df_nodes_2.loc[df_nodes_2['Id'].isin(ls_for_1),'type3']='for1'
# df_nodes_2.loc[df_nodes_2['Id'].isin(ls_for_2),'type3']='for2'
# df_nodes_4.to_csv(r'E:\project_data\18changsha\localgraph_nodes13.csv',index=False)
# df_nodes_2.to_csv(r'E:\project_data\18changsha\forcastgraph_nodes13.csv',index=False)

In [55]:
df_nodes_4['type3'].unique().tolist()

['for2', 'for1', 'none', 'local_best']

In [40]:
df_ns_gz.loc[:,['ind_g','rca']]

,ind_g,rca
6685,929,1.575265
6686,932,1.733672
6777,2212,2.607792
6805,2469,2.552633
6821,2651,1.429139
...,...,...
7405,8436,2.564688
7410,8610,3.861851
7412,8710,4.165270
7413,8720,2.628036


In [38]:
df_forcast_gz

,t,origin,destination,direction
0,1,1921,2919,上游
1,1,1929,2919,上游
2,1,2330,8020,上游
3,1,2330,3940,下游
4,1,2419,3479,下游
...,...,...,...,...
432,2,7451,5941,上游
433,2,7451,5930,上游
434,2,7451,7910,下游
435,2,7451,5255,下游


In [25]:
df_forcast_gz

,t,origin,destination,direction
0,1,1921,2919,上游
1,1,1929,2919,上游
2,1,2330,8020,上游
3,1,2330,3940,下游
4,1,2419,3479,下游
...,...,...,...,...
432,2,7451,5941,上游
433,2,7451,5930,上游
434,2,7451,7910,下游
435,2,7451,5255,下游


In [12]:
df_gz

,area_city,name_city,行业代码,name_ind,企业数量,city_num,ind_num,total_num,rca
44960,4401,广州市,C1789,其他产业用纺织制成品制造,416,1265090,9411,38783772,1.355146
46083,4401,广州市,C1830,服饰制造,10328,1265090,119279,38783772,2.654489
46714,4401,广州市,C1921,皮革服装制造,1344,1265090,8914,38783772,4.622269
47010,4401,广州市,C1922,皮箱、包(袋)制造,4624,1265090,26661,38783772,5.317041
47255,4401,广州市,C1923,皮手套及皮装饰制品制造,166,1265090,2102,38783772,2.421051
...,...,...,...,...,...,...,...,...,...
310203,4401,广州市,C3842,镍氢电池制造,3,1265090,56,38783772,1.642335
314690,4401,广州市,Q8436,计划生育技术服务活动,20,1265090,121,38783772,5.067261
314755,4401,广州市,S9100,中国共产党机关,50,1265090,612,38783772,2.504651
317767,4401,广州市,Q8433,妇幼保健院(所、站),1449,1265090,1552,38783772,28.622350


In [4]:
# nansha
df_total4=pd.read_csv(r'E:\project_data\22nansha\tupu\gz_all_dis_rca.csv')
df_cs=df_total4.loc[(df_total4['rca']>=1.3)&(df_total4['area_city']=='4301')]
ls_best=df_cs['行业代码'].unique().tolist()

,Source,Target,Type,Id,Label,timeset,Weight
0,1019,6010,Undirected,0,NaN,NaN,0.814815
1,411,1369,Undirected,1,NaN,NaN,0.808511
2,5941,7297,Undirected,2,NaN,NaN,0.806818
3,4720,9052,Undirected,3,NaN,NaN,0.806452
4,4720,7910,Undirected,4,NaN,NaN,0.797872
...,...,...,...,...,...,...,...
2754,3531,9541,Undirected,2754,NaN,NaN,0.200000
2755,2456,9542,Undirected,2755,NaN,NaN,0.300000
2756,7716,9610,Undirected,2756,NaN,NaN,0.602564
2757,1743,9620,Undirected,2757,NaN,NaN,0.370370


In [27]:
df_total1=pd.read_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\1990_2020_2022.csv',dtype={'area_city':'str'})
df_total2=df_total1.loc[(df_total1['年份']==2022)&(df_total1['area_city'].isin(ls_city_code))&(df_total1['行业代码'].isin(ls_ind4))]
df_total2['企业数量'].sum()

38783772

In [36]:
df_total21=df_total2.groupby(['area_city','行业代码'])['企业数量'].sum().reset_index()
df_total211=df_total21.groupby(['area_city'])['企业数量'].sum().reset_index()
df_total211=df_total211.rename(columns={'企业数量':'city_num'})
df_total212=df_total21.groupby(['行业代码'])['企业数量'].sum().reset_index()
df_total212=df_total212.rename(columns={'企业数量':'ind_num'})
total=df_total21['企业数量'].sum()
df_total3=pd.merge(left=df_total21,right=df_total211,on='area_city',how='outer')
df_total4=pd.merge(left=df_total3,right=df_total212,on='行业代码',how='outer')
df_total4.loc[:,'total_num']=total
df_total4.loc[:,'rca']=(df_total4['企业数量']/df_total4['city_num'])/(df_total4['ind_num']/df_total4['total_num'])

In [38]:
df_total4.to_csv(r'E:\project_basedata\2023090601启信宝企业统计数据\2022rca.csv',index=False)

In [94]:
# 长沙4301
df_cs=df_total4.loc[(df_total4['rca']>=1.3)&(df_total4['area_city']=='4301')]
ls_best=df_cs['行业代码'].unique().tolist()
df_net_new=pd.read_csv(r'E:\project_data\18changsha\graph.csv',dtype={'Weight':'float'})
df_nodes=pd.read_csv(r'E:\project_data\18changsha\nodes.csv')
df_nodes_1=df_nodes.loc[:,['Id','Label','type1']]
print(len(ls_best))
df=pd.DataFrame(ls_best,columns=['Id'])
print(df.loc[df['Id'].str.contains('27')])
ls_local=df_total4.loc[(df_total4['area_city']=='4301')]['行业代码'].unique().tolist()
ls_local_int=[int(x[1:]) for x in ls_local]
ls_local_best=[int(x[1:]) for x in ls_best]

221
       Id
14  C2762
47  F5127


In [92]:
df_cs_normal=df_total4.loc[(df_total4['area_city']=='4301')]
df_cs_normal=df_cs_normal.copy()
df_cs_normal.loc[:,'Id']=df_cs_normal['行业代码'].str[1:]
df_cs_normal.loc[:,'Id']=df_cs_normal['Id'].astype('int')
df_cs1=df_cs_normal.loc[:,['Id','rca']]

In [73]:
threshold = df_net_new['Weight'].quantile(0.3)
threshold

0.676056

In [95]:
code_ls=[int(x[1:]) for x in ls_best]
ls_forcast=simulate(code_ls,df_net_new,threshold)
df_forcast=pd.DataFrame(ls_forcast,columns=['t','o','d','direction'])
# df_forcast_2=df_forcast.loc[df_forcast['t']<=2]
ls_for_1=df_forcast.loc[df_forcast['t']==1]['d'].unique().tolist()
ls_for_2=df_forcast.loc[df_forcast['t']==2]['d'].unique().tolist()

df_nodes_3=df_nodes_1.copy()
df_nodes_3.loc[df_nodes_3['Id'].isin(ls_local_int),'type2']='local_normal'
df_nodes_3.loc[df_nodes_3['Id'].isin(ls_local_best),'type2']='local_best'
df_nodes_3.loc[df_nodes_3['type2'].isnull(),'type2']='none'
df_nodes_4=pd.merge(left=df_nodes_3,right=df_cs1,on=['Id'],how='left')
df_nodes_4.loc[df_nodes_4['rca'].isnull(),'rca']=0
df_nodes_2=df_nodes_1.copy()
df_nodes_2.loc[df_nodes_2['Id'].isin(ls_local_best),'type3']='local_best'
df_nodes_2.loc[df_nodes_2['Id'].isin(ls_for_1),'type3']='for1'
df_nodes_2.loc[df_nodes_2['Id'].isin(ls_for_2),'type3']='for2'
df_nodes_4.to_csv(r'E:\project_data\18changsha\localgraph_nodes13.csv',index=False)
df_nodes_2.to_csv(r'E:\project_data\18changsha\forcastgraph_nodes13.csv',index=False)

In [96]:
len(code_ls)

221

In [52]:
df=pd.DataFrame(ls_best,columns=['Id'])
df.loc[df['Id'].str.contains('27')]

,Id
4,C2762


In [33]:
# 读取施河镇企业数据
df_shihe=pd.read_excel(r'E:\project_项目\14施河镇\项目过程\施河镇企业行业数据.xlsx',dtype='str')
df_shihe_group= df_shihe.groupby(['小类'])['统一社会信用代码'].count().reset_index()
df_shihe_group.loc[:,'district_code']='shihe'
df_shihe_group.rename(columns={'小类':'industry_code','统一社会信用代码':'count'},inplace=True)
df_total = pd.read_csv(r'E:\project_tupu\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_total_1=pd.concat([df_total,df_shihe_group],axis=0)
# 筛选各区县小类企业数据
df_ind = pd.read_excel(r'E:\project_basedata\industry_new.xlsx',dtype='str')
df_ind=df_ind.rename(columns={'小类代码':'Id'}).copy()
ls_ind_4=df_ind['小类类别代码'].unique().tolist()
# 读取全国地区代码
# df_dis_1=pd.read_excel(r'E:\project_tupu\districtcode(1).xlsx',dtype='str')
df_dis_2=pd.read_excel(r'E:\project_basedata\t_admin_division_code.xlsx',dtype='str')
# ls_df_dis_1=df_dis_1['code3'].unique().tolist()
ls_df_dis_2=df_dis_2['type_code'].unique().tolist()

df_total_filt=df_total_1.loc[(df_total_1['district_code'].isin(ls_df_dis_2))&(df_total_1['industry_code'].isin(ls_ind_4))]
df_total_filt_1=df_total_filt.copy()
df_total_filt_1=pd.concat([df_total_filt_1,df_shihe_group],axis=0)
df_dis_num=df_total_filt_1.groupby(['district_code'])['count'].sum().reset_index()
df_dis_num.rename(columns={'count':'count_dis'},inplace=True)
df_total_dis=pd.merge(left=df_total_filt_1,right=df_dis_num,on=['district_code'],how='outer')
df_ind_num=df_total_filt_1.groupby(['industry_code'])['count'].sum().reset_index()
df_ind_num.rename(columns={'count':'count_ind'},inplace=True)
total_num_1=df_ind_num['count_ind'].sum()
total_num_2=df_dis_num['count_dis'].sum()
df_connect=pd.read_csv(r'E:\project_tupu\ind_connect_0705.csv',dtype='str')
df_connect=df_connect.astype(dtype={'Weight':'float'})
threshold = df_connect['Weight'].quantile(0.2)

if total_num_1==total_num_2:
    df_total_dis_ind=pd.merge(left=df_total_dis,right=df_ind_num,on=['industry_code'],how='outer')
    df_total_dis_ind.loc[:,'total_num']=total_num_2
    df_total_dis_ind.loc[:,'rca']=(df_total_dis_ind['count']/df_total_dis_ind['count_dis'])/(df_total_dis_ind['count_ind']/df_total_dis_ind['total_num'])
    ls_four=['shihe']
    for i in ls_four:
        df_i=df_total_dis_ind.loc[df_total_dis_ind['district_code']==i]
        file_path='E:\\project_data\\14shihezhen\\'+i+'_adv.csv'
        df_i.to_csv(file_path,index=False,mode='w')
        df_i.loc[:,'ind_num']=df_i['industry_code'].str[1:]
        df_i1=df_i.astype(dtype={'ind_num':'int'}).copy()
        df_i=df_i1.astype(dtype={'ind_num':'str'}).copy()
        code_ls=df_i.loc[df_i['rca']>=2]['ind_num'].unique().tolist()
        
        ls_result=simulate(code_ls,df_connect,threshold)
        df_i_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
        df_forcast_ = df_i_result.loc[df_i_result['t'] <= 2]
        file_path_i_result='E:\\project_data\\14shihezhen\\'+i+'_ind_predict.csv'
        df_forcast_.to_csv(file_path_i_result,index=False,mode='w')
        
        ls_local=df_i['ind_num'].unique().tolist()
        ls1 = df_connect['Source'].unique().tolist()
        ls2 = df_connect['Target'].unique().tolist()
        ls = ls1+ls2
        ls = list(Counter(ls))
        df_nodes = pd.DataFrame(ls,columns = ['Id'])
        df_best = df_forcast_.loc[df_forcast_['t'] == 1]
        ls_best = df_best['origin'].unique().tolist()
        trans_frt = df_best['destination'].unique().tolist()
        trans_frt_1 = [x for x in trans_frt if x not in ls_local]
        trans_frt_2 = [x for x in trans_frt if x in ls_local]
        trans_sec = df_forcast_.loc[df_forcast_['t'] == 2]['destination'].unique().tolist()
        trans_sec_1 = [x for x in trans_sec if x not in ls_local]
        trans_sec_2 = [x for x in trans_sec if x in ls_local]
        df_nodes.loc[df_nodes['Id'].isin(ls_best),'tag'] = 'best'
        df_nodes.loc[df_nodes['Id'].isin(code_ls),'tag_ori'] = 'best'
        df_nodes.loc[df_nodes['Id'].isin(trans_frt_1),'tag'] = 'trans_frt'
        df_nodes.loc[df_nodes['Id'].isin(trans_frt_2),'tag'] = 'local_frt'
        df_nodes.loc[df_nodes['Id'].isin(trans_sec_1),'tag'] = 'trans_sec'
        df_nodes.loc[df_nodes['Id'].isin(trans_sec_2),'tag'] = 'local_sec'
        df_result_nodes = pd.merge(left = df_nodes, right = df_ind,on = ['Id'],how = 'left')
        df_result_nodes_1=df_result_nodes.loc[:,['Id','小类名称','tag','tag_ori']]
        df_result_nodes_1=df_result_nodes_1.rename(columns={'小类名称':'Label'})
        
        file_path_i_result_1='E:\\project_data\\14shihezhen\\'+i+'_ind_nodes.csv'
        df_result_nodes_1.to_csv(file_path_i_result_1,index=False,mode='w')

C:\Users\duanyousheng\AppData\Local\Temp\ipykernel_21348\1096214935.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_i.loc[:,'ind_num']=df_i['industry_code'].str[1:]


In [31]:
df_result_nodes

,Id,tag,tag_ori,门类代码,门类名称,大类代码,大类名称,中类代码,中类名称,小类类别代码,小类类别名称,小类类别代码.1,小类名称
0,1019,trans_sec,NaN,B0000,采矿业,B1000,非金属矿采选业,B1010,土砂石开采,B1019,粘土及其他土砂石开采,B1019,粘土及其他土砂石开采
1,411,trans_sec,NaN,A0000,农、林、牧、渔业,A0400,渔业,A0410,水产养殖,A0411,海水养殖,A0411,海水养殖
2,5941,trans_frt,NaN,G0000,交通运输、仓储和邮政业,G5900,装卸搬运和仓储业,G5940,危险品仓储,G5941,油气仓储,G5941,油气仓储
3,4720,best,best,E0000,建筑业,E4700,房屋建筑业,E4720,体育场馆建筑,E4720,体育场馆建筑,E4720,体育场馆建筑
4,3990,NaN,NaN,C0000,制造业,C3900,计算机、通信和其他电子设备制造业,C3990,其他电子设备制造,C3990,其他电子设备制造,C3990,其他电子设备制造
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,9541,NaN,NaN,S0000,公共管理、社会保障和社会组织,S9500,群众团体、社会团体和其他成员组织,S9540,宗教组织,S9541,宗教团体服务,S9541,宗教团体服务
1373,9542,NaN,NaN,S0000,公共管理、社会保障和社会组织,S9500,群众团体、社会团体和其他成员组织,S9540,宗教组织,S9542,宗教活动场所服务,S9542,宗教活动场所服务
1374,9610,NaN,NaN,S0000,公共管理、社会保障和社会组织,S9600,基层群众自治组织及其他组织,S9610,社区居民自治组织,S9610,社区居民自治组织,S9610,社区居民自治组织
1375,9620,NaN,NaN,S0000,公共管理、社会保障和社会组织,S9600,基层群众自治组织及其他组织,S9620,村民自治组织,S9620,村民自治组织,S9620,村民自治组织


In [26]:
# 读取全国行业数据
df_total = pd.read_csv(r'E:\project_tupu\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_ind = pd.read_excel(r'E:\project_basedata\industry_new.xlsx',dtype='str')
df_ind['ind4']=df_ind['小类代码'].str[1:]
ls_ind_4=df_ind['小类类别代码'].unique().tolist()

# 读取全国地区代码
# df_dis_1=pd.read_excel(r'E:\project_tupu\districtcode(1).xlsx',dtype='str')
df_dis_2=pd.read_excel(r'E:\project_basedata\t_admin_division_code.xlsx',dtype='str')
# ls_df_dis_1=df_dis_1['code3'].unique().tolist()
ls_df_dis_2=df_dis_2['type_code'].unique().tolist()

df_total_filt=df_total.loc[(df_total['district_code'].isin(ls_df_dis_2))&(df_total['industry_code'].isin(ls_ind_4))]
# 提取行业中类信息
df_total_filt_1=df_total_filt.copy()
df_total_filt_1.loc[:,'ind_3']=df_total_filt['industry_code'].str[:4]
df_total_filt_2=df_total_filt_1.groupby(['district_code','ind_3'])['count'].sum().reset_index()

# 计算各地各行业rca
df_dis_num=df_total_filt_2.groupby(['district_code'])['count'].sum().reset_index()
df_dis_num.rename(columns={'count':'count_dis'},inplace=True)
df_total_dis=pd.merge(left=df_total_filt_2,right=df_dis_num,on=['district_code'],how='outer')
df_ind_num=df_total_filt_2.groupby(['ind_3'])['count'].sum().reset_index()
df_ind_num.rename(columns={'count':'count_ind'},inplace=True)
total_num_1=df_ind_num['count_ind'].sum()
total_num_2=df_dis_num['count_dis'].sum()
if total_num_1==total_num_2:
    df_total_dis_ind=pd.merge(left=df_total_dis,right=df_ind_num,on=['ind_3'],how='outer')
    df_total_dis_ind.loc[:,'total_num']=total_num_2
    df_total_dis_ind.loc[:,'rca']=(df_total_dis_ind['count']/df_total_dis_ind['count_dis'])/(df_total_dis_ind['count_ind']/df_total_dis_ind['total_num'])
    # 呼和浩特，包头，鄂尔多斯，榆林城市分别
    # ls_four=['150100','150200','150600','610800']
    ls_four=['150600']
    for i in ls_four:
        df_i=df_total_dis_ind.loc[df_total_dis_ind['district_code']==i]
        file_path='E:\\project_data\\13eerduosi\\hubaoeyu\\'+i+'_adv_ind3.csv'
        df_i.to_csv(file_path,index=False,mode='w')

In [2]:
df_total = pd.read_csv(r'E:\project_tupu\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_total

,district_code,industry_code,count
0,540230,N7690,1
1,350302,C3554,1
2,450329,B0931,1
3,640402,B1011,1
4,350504,C1783,2
...,...,...,...
2163116,370402,J6650,1
2163117,130181,C2829,3
2163118,510302,F5227,43
2163119,130200,C4011,1


In [53]:
# 读取全国行业数据
df_total = pd.read_csv('F:\\tupu\\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_ind = pd.read_excel(r'F:\tupu\industry_new.xlsx',dtype='str')
df_ind['ind4']=df_ind['小类代码'].str[1:]
ls_ind_4=df_ind['小类代码'].unique().tolist()

# 读取全国地区代码
df_dis_1=pd.read_excel(r'F:\tupu\districtcode(1).xlsx',dtype='str')
df_dis_2=pd.read_excel(r'F:\tupu\t_admin_division_code(1).xlsx',dtype='str')
ls_df_dis_1=df_dis_1['code3'].unique().tolist()
ls_df_dis_2=df_dis_2['type_code'].unique().tolist()

df_total_filt=df_total.loc[(df_total['district_code'].isin(ls_df_dis_2))&(df_total['industry_code'].isin(ls_ind_4))]
# 提取行业中类信息
df_total_filt.loc[:,'ind_3']=df_total_filt['industry_code'].str[:3]
# 计算各地各行业rca
df_dis_num=df_total_filt.groupby(['district_code'])['count'].sum().reset_index()
df_dis_num.rename(columns={'count':'count_dis'},inplace=True)
df_total_dis=pd.merge(left=df_total_filt,right=df_dis_num,on=['district_code'],how='outer')
df_ind_num=df_total_filt.groupby(['industry_code'])['count'].sum().reset_index()
df_ind_num.rename(columns={'count':'count_ind'},inplace=True)
total_num_1=df_ind_num['count_ind'].sum()
total_num_2=df_dis_num['count_dis'].sum()
if total_num_1==total_num_2:
    df_total_dis_ind=pd.merge(left=df_total_dis,right=df_ind_num,on=['industry_code'],how='outer')
    df_total_dis_ind.loc[:,'total_num']=total_num_2
    df_total_dis_ind.loc[:,'rca']=(df_total_dis_ind['count']/df_total_dis_ind['count_dis'])/(df_total_dis_ind['count_ind']/df_total_dis_ind['total_num'])
    # 呼和浩特，包头，鄂尔多斯，榆林城市分别
    ls_four=['150100','150200','150600','610800']
    for i in ls_four:
        df_i=df_total_dis_ind.loc[(df_total_dis_ind['district_code']==i)&(df_total_dis_ind['rca']>=1.7)]
        file_path='F:\\tupu\\'+i+'_adv_ind.csv'
        df_i.to_csv(file_path,index=False,mode='w')
    
    df_total_dis_ind_adv_four=df_total_dis_ind.loc[(df_total_dis_ind['district_code'].isin(ls_four))&(df_total_dis_ind['rca']>=1.7)]
    df_total_dis_ind_adv_four_group=df_total_dis_ind_adv_four.groupby(['industry_code'])['district_code'].count().reset_index()
    df_total_dis_ind_adv_four_group=df_total_dis_ind_adv_four_group.rename(columns={'district_code':'district_count'})
    df_four_city_result=pd.merge(left=df_total_dis_ind_adv_four_group,right=df_total_dis_ind_adv_four,on=['industry_code'],how='outer')
    df_four_city_result_1=df_four_city_result.groupby(['industry_code'])['district_code'].sum().reset_index()
    df_four_city_result_1=df_four_city_result_1.rename(columns={'district_code':'district_name'})
    df_four_city_result_2=pd.merge(left=df_four_city_result_1,right=df_four_city_result,on=['industry_code'],how='left')
    file_path_1='F:\\tupu\\all_four_adv_ind.csv'
    df_four_city_result_2.to_csv(file_path_1,index=False,mode='w')

In [55]:
# 呼和浩特，包头，鄂尔多斯，榆林城市合并
# ['150100','150200','150600','610800']
# 四个城市一起求rca
ls_four=['150100','150200','150600','610800']
df_total_four=df_total_filt.loc[df_total_filt['district_code'].isin(ls_four)]
df_total_nofour=df_total_filt.loc[~(df_total_filt['district_code'].isin(ls_four))]
df_total_four_new=df_total_four.groupby(['industry_code'])['count'].sum().reset_index()
df_total_four_new.loc[:,'district_code']='hubaoeryu'
df_total_four_new=df_total_four_new.loc[:,['district_code','industry_code','count']]
df_total_filt_1=pd.concat([df_total_nofour,df_total_four_new],axis=0)

df_dis_num=df_total_filt_1.groupby(['district_code'])['count'].sum().reset_index()
df_dis_num.rename(columns={'count':'count_dis'},inplace=True)
df_total_dis=pd.merge(left=df_total_filt_1,right=df_dis_num,on=['district_code'],how='outer')
df_ind_num=df_total_filt_1.groupby(['industry_code'])['count'].sum().reset_index()
df_ind_num.rename(columns={'count':'count_ind'},inplace=True)
total_num_1=df_ind_num['count_ind'].sum()
total_num_2=df_dis_num['count_dis'].sum()
if total_num_1==total_num_2:
    df_total_dis_ind=pd.merge(left=df_total_dis,right=df_ind_num,on=['industry_code'],how='outer')
    df_total_dis_ind.loc[:,'total_num']=total_num_2
    df_total_dis_ind.loc[:,'rca']=(df_total_dis_ind['count']/df_total_dis_ind['count_dis'])/(df_total_dis_ind['count_ind']/df_total_dis_ind['total_num'])
# 按照rca阈值2进行数据筛选
df_total_dis_ind_adv=df_total_dis_ind.loc[(df_total_dis_ind['district_code']=='hubaoeryu')&(df_total_dis_ind['rca']>=1.7)]
file_path_1='F:\\tupu\\four_city_together_adv_ind.csv'
df_total_dis_ind_adv.to_csv(file_path_1,index=False,mode='w')

In [62]:
# 图谱预测
# 联系数据读取
# 读取全国行业数据
df_total = pd.read_csv('F:\\tupu\\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_ind = pd.read_excel(r'F:\tupu\industry_new.xlsx',dtype='str')
ls_ind_4=df_ind['小类代码'].unique().tolist()

# 读取全国地区代码
df_dis_1=pd.read_excel(r'F:\tupu\districtcode(1).xlsx',dtype='str')
df_dis_2=pd.read_excel(r'F:\tupu\t_admin_division_code(1).xlsx',dtype='str')
ls_df_dis_1=df_dis_1['code3'].unique().tolist()
ls_df_dis_2=df_dis_2['type_code'].unique().tolist()

df_total_filt=df_total.loc[(df_total['district_code'].isin(ls_df_dis_2))&(df_total['industry_code'].isin(ls_ind_4))]

df_connect=pd.read_csv(r'F:\tupu\ind_connect_0705.csv',dtype='str')
df_connect=df_connect.astype(dtype={'Weight':'float'})
df_ind = pd.read_excel(r'F:\tupu\industry_new.xlsx',dtype='str')
df_total_filt.loc[:,'ind_num']=df_total_filt['industry_code'].str[1:]
df_total_filt=df_total_filt.astype(dtype={'ind_num':'int'},inplace=True)
df_total_filt=df_total_filt.astype(dtype={'ind_num':'str'},inplace=True)
ls_district=os.listdir(r'F:\tupu')
threshold = df_connect['Weight'].quantile(0.2)
ls_four=['150100','150200','150600','610800']
for i in ls_district:
    if i.endswith('_adv_ind.csv'):
        file_path_i='F:\\tupu\\'+i
        location=i[:-12]
        df_i=pd.read_csv(file_path_i)
        df_i.loc[:,'ind_num']=df_i['industry_code'].str[1:]
        df_i=df_i.astype(dtype={'ind_num':'int'},inplace=True)
        df_i=df_i.astype(dtype={'ind_num':'str'},inplace=True)
        code_ls=df_i['ind_num'].unique().tolist()
        ls_result=simulate(code_ls,df_connect,threshold)
        df_i_result=pd.DataFrame(ls_result,columns=['t','origin','destination','direction'])
        df_forcast_ = df_i_result.loc[df_i_result['t'] <= 2]
        file_path_i_result='F:\\tupu\\predict_'+location+'.csv'
        df_forcast_.to_csv(file_path_i_result,index=False,mode='w')
        
        if len(location)!=6:
            ls_local=df_total_filt.loc[df_total_filt['district_code'].isin(ls_four)]['ind_num'].unique().tolist()
        else:
            ls_local=df_total_filt.loc[df_total_filt['district_code']==location]['ind_num'].unique().tolist()
        
        ls1 = df_connect['Source'].unique().tolist()
        ls2 = df_connect['Target'].unique().tolist()
        ls = ls1+ls2
        ls = list(Counter(ls))
        df_nodes = pd.DataFrame(ls,columns = ['Id'])
        df_best = df_forcast_.loc[df_forcast_['t'] == 1]
        ls_best = df_best['origin'].unique().tolist()
        trans_frt = df_best['destination'].unique().tolist()
        trans_frt_1 = [x for x in trans_frt if x not in ls_local]
        trans_frt_2 = [x for x in trans_frt if x in ls_local]
        trans_sec = df_forcast_.loc[df_forcast_['t'] == 2]['destination'].unique().tolist()
        trans_sec_1 = [x for x in trans_sec if x not in ls_local]
        trans_sec_2 = [x for x in trans_sec if x in ls_local]
        df_nodes.loc[df_nodes['Id'].isin(ls_best),'tag'] = 'best'
        df_nodes.loc[df_nodes['Id'].isin(code_ls),'tag_ori'] = 'best'
        df_nodes.loc[df_nodes['Id'].isin(trans_frt_1),'tag'] = 'trans_frt'
        df_nodes.loc[df_nodes['Id'].isin(trans_frt_2),'tag'] = 'local_frt'
        df_nodes.loc[df_nodes['Id'].isin(trans_sec_1),'tag'] = 'trans_sec'
        df_nodes.loc[df_nodes['Id'].isin(trans_sec_2),'tag'] = 'local_sec'
        df_result_nodes = pd.merge(left = df_nodes, right = df_ind,on = ['Id'],how = 'left')
        df_result_nodes_1=df_result_nodes.loc[:,['Id','Label','tag','tag_ori']]
        if 'all_four' in i:
            df_result_nodes_2=pd.merge(left=df_result_nodes_1,right=df_i.loc[:,['ind_num','district_name','district_count','rca']],
                                       left_on=['Id'],right_on=['ind_num'],how='left')
            df_result_nodes_3=df_result_nodes_2.loc[:,['Id','Label','tag','tag_ori','district_name','district_count','rca']]
        else:
            df_result_nodes_2=pd.merge(left=df_result_nodes_1,right=df_i.loc[:,['ind_num', 'rca']],
                                       left_on=['Id'],right_on=['ind_num'],how='left')
            df_result_nodes_3=df_result_nodes_2.loc[:,['Id','Label','tag','tag_ori','rca']]
        file_path_i_result_1='F:\\tupu\\predict_nodes_'+location+'.csv'
        df_result_nodes_3.to_csv(file_path_i_result_1,index=False,mode='w')

C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Lenovo\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
df_sk=pd.read_

In [61]:
# 数库和国标对应
def sk_to_gb(df_sk,df_gb,code,name):
    

,ind_num,rca
0,4899,2.152350
1,1722,1.970900
2,1721,4.564861
3,6110,10.032901
4,5219,1.851097
5,7259,2.465582
6,6319,4.063730
7,4430,5.374269
8,7010,2.963687
9,7249,2.225645


In [31]:
for i in ls_district:
    if i.endswith('_adv_ind.csv'):
        file_path_i='F:\\tupu\\'+i
        location=i[:-12]
        print(location)

150100
150200
150600
610800
all_four
four_city_together


In [ ]:
ls_local = df_ent['Id'].tolist()
ls1 = df_forcast_2['origin'].unique().tolist()
ls2 = df_forcast_2['destination'].unique().tolist()
ls = ls1+ls2
ls = list(Counter(ls))
df_nodes = pd.DataFrame(ls,columns = ['Id'])
df_best = df_forcast_2.loc[df_forcast_2['t'] == 1]
ls_best = df_best['origin'].unique().tolist()
trans_frt = df_best['destination'].unique().tolist()
trans_frt_1 = [x for x in trans_frt if x not in ls_local]
trans_frt_2 = [x for x in trans_frt if x in ls_local]
trans_sec = df_forcast_2.loc[df_forcast_2['t'] == 2]['destination'].unique().tolist()
trans_sec_1 = [x for x in trans_sec if x not in ls_local]
trans_sec_2 = [x for x in trans_sec if x in ls_local]
df_nodes.loc[df_nodes['Id'].isin(ls_best),['tag']] = 'best'
df_nodes.loc[df_nodes['Id'].isin(trans_frt_1),['tag']] = 'trans_frt'
df_nodes.loc[df_nodes['Id'].isin(trans_frt_2),['tag']] = 'local_frt'
df_nodes.loc[df_nodes['Id'].isin(trans_sec_1),['tag']] = 'trans_sec'
df_nodes.loc[df_nodes['Id'].isin(trans_sec_2),['tag']] = 'local_sec'
df_result = pd.merge(left = df_nodes, right = df_ent,on = ['Id'],how = 'outer')
df_result.loc[df_result['tag_x'].isnull(),['tag_x']] = 'local_normal'

In [30]:
df_net_new=df_connect
threshold = df_connect['Weight'].quantile(0.6)
file_path_i='F:\\tupu\\'+'four_city_together_adv_ind.csv'
df_i=pd.read_csv(file_path_i)
df_i.loc[:,'ind_num']=df_i['industry_code'].str[1:]
df_i=df_i.astype(dtype={'ind_num':'int'},inplace=True)
code_ls=df_i['ind_num'].unique().tolist()
len(simulate(code_ls,df_net_new,threshold))

301

In [14]:
df_connect['Weight'].quantile(0.3)

0.676056

In [20]:
#ls=[10,9,8,7,6,5,4,3,2,1]
ls=[1,2,3,4,5,6,7,8,9,10]
pd.DataFrame(ls)[0].quantile(0.3)

3.6999999999999997

In [13]:
df_connect

,Source,Target,Type,Id,Label,timeset,Weight
0,1019,6010,Undirected,0,NaN,NaN,0.814815
1,411,1369,Undirected,1,NaN,NaN,0.808511
2,5941,7297,Undirected,2,NaN,NaN,0.806818
3,4720,9052,Undirected,3,NaN,NaN,0.806452
4,4720,7910,Undirected,4,NaN,NaN,0.797872
5,3990,8433,Undirected,5,NaN,NaN,0.793103
6,5941,5942,Undirected,6,NaN,NaN,0.792208
7,7241,8760,Undirected,7,NaN,NaN,0.788889
8,4720,6991,Undirected,8,NaN,NaN,0.788889
9,7910,9052,Undirected,9,NaN,NaN,0.787234


In [4]:
df_total = pd.read_csv('E:\\project_tupu\\202201.csv',dtype ={'district_code':'str','industry_code':'str','count':'int'})
df_ind = pd.read_excel('E:\\project_basedata\\industry_new.xlsx')
df_total['ind']=df_total['industry_code'].str[1:]
ls_ind4 = df_ind['小类代码.1'].tolist()
df_total_new = df_total.loc[df_total['industry_code'].isin(ls_ind4)]
df_total_new

,district_code,industry_code,count,ind
0,540230,N7690,1,7690
1,350302,C3554,1,3554
2,450329,B0931,1,0931
3,640402,B1011,1,1011
4,350504,C1783,2,1783
...,...,...,...,...
2163116,370402,J6650,1,6650
2163117,130181,C2829,3,2829
2163118,510302,F5227,43,5227
2163119,130200,C4011,1,4011


In [11]:
# 康巴什企业数据
df_kbs = pd.read_csv('E:\\project_data\\10kangbashi\\t_enterprise_kangba_fin.csv',dtype = 'str')
df_kbs_new = df_kbs.dropna(subset = ['small_industry_code'])
df_kbs_new_1 = df_kbs_new.loc[df_kbs_new['small_industry_code'].isin(ls_ind4)]

In [12]:
df_kbs_new_1

,year,name,small_industry_code,start_date,regist_capi_new
20,2000,中国银行股份有限公司鄂尔多斯市康巴什支行,J6610,1994-01-07,0.0
21,2000,内蒙古电力（集团）有限责任公司鄂尔多斯电业局大用户管理处,D4420,1998-04-28,0.0
26,2000,鄂尔多斯市天皓水泥有限公司,C3021,1998-10-16,7000.0
33,2000,鄂尔多斯市神华神东房地产开发有限责任公司,K7010,2000-06-04,2000.0
34,2000,鄂尔多斯市智慧能源科技有限公司,M7484,1999-04-30,2727.2727
...,...,...,...,...,...
36684,1999,鄂尔多斯市智慧能源科技有限公司,M7484,1999-04-30,2727.2727
36685,1999,鄂尔多斯市泰达投资发展有限责任公司,G5429,1999-04-02,5000.0
36687,1999,鄂尔多斯市党校教学服务有限公司,H6190,1992-09-16,39.0
36694,1999,中国农业银行股份有限公司鄂尔多斯康巴什新区正阳街支行,J6629,1994-01-08,0.0


In [12]:
# 北戴河企业数据
df_beidaihe = pd.read_csv('E:\\project_data\\03beidaihe\\data\\hebei_xc_industry1_1.csv',dtype = 'str',encoding = 'gbk')
df_beidaihe_new = df_beidaihe.loc[df_beidaihe['industry_code'].isin(ls_ind4)]
df_beidaihe_new

,eid,name,longitude,latitude,mid_industry_code,small_industry_code,small_name,start_date,industry_code,regist_capi_new
0,931cc293-ef68-4bce-8fdd-52cf35f71a32,秦皇岛抚宁区松达种植专业合作社,119.33552705495863,39.82077665236091,A014,A0141,蔬菜种植,2018-06-25,A0141,3.0
1,53035c4c-4626-44a5-90be-f4c6b1493bca,昌黎县德升果蔬种植专业合作社,119.11902383891287,39.57156584484489,A015,A0151,仁果类和核果类水果种植,2019-06-20,A0151,50.0
2,b5633718-14dd-4f72-b9fb-86f722ea6e00,秦皇岛然冲环保科技有限公司,119.392008283,39.77408538,M751,M7516,环保技术推广服务,2020-10-19,M7516,100.0
3,10e638e1-f20b-46bd-a41f-12b3c7d90072,中国联合网络通信有限公司昌黎县分公司安山营业厅,118.996687834,39.720590181,I631,I6311,固定电信服务,2008-11-14,I6311,0.0
4,37a32945-9ae4-44eb-8e6b-87dea8aa1273,昌黎县通飞商贸有限公司,119.170256268,39.713218221,F513,F5131,纺织品、针织品及原料批发,2020-06-01,F5131,1000.0
...,...,...,...,...,...,...,...,...,...,...
12634,a9a33f8b-2e79-41c9-bdd9-c6249b338b26,秦皇岛宗宝水果种植专业合作社,119.20889418484475,39.82331140033323,A015,A0151,仁果类和核果类水果种植,2016-11-25,A0151,30.0
12635,8ffb4fc7-086f-4650-954e-2c21df115e77,昌黎县佰岳商贸有限公司,118.996687834,39.720590181,F512,F5127,酒、饮料及茶叶批发,2019-05-28,F5127,50.0
12636,cfb5698a-31a7-475c-b02f-eb3202fc4d20,昌黎县金手指会计服务有限公司,119.18923997893084,39.71862910136542,L724,L7241,会计、审计及税务服务,2019-03-05,L7241,50.0
12637,95c88bad-0c34-45e8-a5c1-15dd2f311473,中国联合网络通信有限公司秦皇岛市抚宁区分公司南戴河中心营业厅,119.430267834,39.795795978,I631,I6311,固定电信服务,2008-11-20,I6311,0.0


In [14]:
ls1 = df_kbs_new_1['small_industry_code'].tolist()
df_v = pd.DataFrame(ls1,columns=['Id'])
df_rca = df_v['Id'].value_counts().reset_index().rename(columns = {'index':'Id','Id':'count_bdh'})
threshold = df_rca['count_bdh'].quantile(0.1)
count_bdh_total=df_rca['count_bdh'].sum()
df_rca['count_bdh_total'] = count_bdh_total

total_num = df_total_new['count'].sum()
df_data_total = df_total_new.loc[:,['industry_code','count']]
df_count_total = df_data_total.groupby(by=['industry_code'])['count'].sum().reset_index()
df_count_total_bdh = df_count_total.loc[df_count_total['industry_code'].isin(df_rca['Id'].tolist())]
df_count_total_bdh.loc[:,['count_total']]= total_num

df_rca_result = pd.merge(left = df_rca, right = df_count_total_bdh, left_on = ['Id'], right_on = ['industry_code'], how = 'inner')
df_rca_result['rca_num']=(df_rca_result['count_bdh']/df_rca_result['count_bdh_total'])/(df_rca_result['count']/df_rca_result['count_total'])
# df_rca_result_ = df_rca_result.loc[df_rca_result['count_bdh'] > threshold]

C:\Users\duanyousheng\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [17]:
df_rca_result.to_csv('E:\\project_data\\10kangbashi\\kbs_ind_rca_1.csv',index = False)

In [9]:
# 来宾市企业数据
df_laibin_qiye = pd.read_csv('E:\\project_data\\08laibing\\来宾市企业.csv',dtype = 'str')
df_laibin_qiye_new = df_laibin_qiye.loc[df_laibin_qiye['industry_code'].isin(ls_ind4)]
df_laibin_qiye_new

,eid,name,district_code,start_date,industry_code,scope1,products
1,154646b0-5590-4ddd-bed5-601e9c71ce53,来宾市万佳电子科技有限公司,451302,2014-11-14,F5176,电子科技研发、推广和应用服务；计算机及配件、办公设备及耗材、电子产品、安防监控设备、音响设备...,NaN
2,231a7a6e-29fd-426d-a0d8-7e7e20e5ed36,来宾市鸿存装饰有限公司,451302,2016-06-20,E5012,室内外装饰工程；装饰材料、日用百货、五金交电批发及零售。（依法须经批准的项目，经相关部门批准...,NaN
3,2faadb1a-3b8b-4dd7-8aef-130059f13bd6,来宾市鼎旺农业科技有限公司,451323,2018-10-08,M7519,农业科学技术开发、推广、交流、转让、咨询服务；农作物种植及种苗培育、销售；农副产品加工、销售...,NaN
4,3f545036-975a-4398-a733-5ee040158278,广西来宾润心农业发展有限公司兴宾区蒙村镇大步村服务站,451302,2017-03-15,A0190,农业信息咨询；农业项目投资；农业技术领域内的技术开发，技术服务，技术咨询；农产品展示；化肥、...,NaN
5,452a4e83-beca-484b-b1f9-733599b3efe9,广西国彩商贸有限公司,451302,2017-06-08,F5299,办公用品、劳保用品、五金交电、建筑材料（危险品除外）、电线电缆、家具、消防器材、电力器材、汽...,NaN
...,...,...,...,...,...,...,...
43130,edce5662-89ff-4fd9-ab90-476616187a11,广西国顺电子科技有限公司,451302,2018-01-30,I6511,软件开发、设计服务；电子产品技术开发、技术服务；信息系统集成服务；办公用品、家具、办公设备、...,NaN
43133,fa4aefe3-187d-49af-9ac2-1e77ba71448e,来宾市巨儒电子科技有限公司,451302,2017-02-17,F5176,计算机、办公耗材、办公设备销售。,NaN
43134,facfc1b4-63aa-4386-a2a4-065b6e1d38cb,象州顺应养殖场,451322,2014-01-07,A0313,生猪养殖、销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动。）,NaN
43135,fcc09a11-9a3b-4476-a02f-ea66eee32f1d,象州县顺鑫木材加工厂,451322,2019-10-24,C2012,拼板、板方材加工及销售。（依法须经批准的项目，经相关部门批准后方可开展经营活动。）,NaN


In [24]:
ls1 = df_laibin_qiye_new['industry_code'].tolist()
df_v = pd.DataFrame(ls1,columns=['Id'])
df_rca = df_v['Id'].value_counts().reset_index().rename(columns = {'index':'Id','Id':'count_laibing'})
threshold = df_rca['count_laibing'].quantile(0.1)
count_laibing_total=df_rca['count_laibing'].sum()
df_rca['count_laibing_total'] = count_laibing_total

total_num = df_total_new['count'].sum()
df_data_total = df_total_new.loc[:,['industry_code','count']]
df_count_total = df_data_total.groupby(by=['industry_code'])['count'].sum().reset_index()
df_count_total_laibing = df_count_total.loc[df_count_total['industry_code'].isin(df_rca['Id'].tolist())]
df_count_total_laibing.loc[:,('count_total')]= total_num

df_rca_result = pd.merge(left = df_rca, right = df_count_total_laibing, left_on = ['Id'], right_on = ['industry_code'], how = 'inner')
df_rca_result['rca_num']=(df_rca_result['count_laibing']/df_rca_result['count_laibing_total'])/(df_rca_result['count']/df_rca_result['count_total'])
df_rca_result_ = df_rca_result.loc[df_rca_result['count_laibing'] > threshold]

<ipython-input-24-089715c11223>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_count_total_laibing['count_total'] = total_num


In [1]:
df_rca_result.to_csv('E:\\project_data\\08laibing\\laibing_rca.csv', index = False)
df_rca_result_.to_csv('E:\\project_data\\08laibing\\laibing_rca_.csv', index = False)

NameError: name 'df_rca_result' is not defined

In [2]:
# 来宾专利
df_lb = pd.read_excel('E:\\project_data\\08laibing\\patent\\来宾专利数据.xlsx',dtype = 'str')

In [14]:
df_patent = df_lb['IPC小类'].value_counts().reset_index()
df_patent['IPC小类'].quantile(0.7)
code_ls = df_patent.loc[df_patent['IPC小类'] > 14]['index'].tolist()

In [35]:
df_net_new = pd.read_csv('E:\\project_tupu\\patent\\patent_total.csv',dtype = {'Source':'str','Target':'str','Weight':'float'})
threshold = df_net_new['Weight'].quantile(0.7)
df_nodes = pd.read_csv('E:\\project_tupu\\patent\\patent_total_nodes.csv',dtype = 'str')
code_total_ls = df_nodes['Id'].tolist()
code_ls = [x for x in code_total_ls if x in code_ls]

In [36]:
print(threshold,len(code_ls))

198.203514 115


In [12]:
ls_name = []
for i in df_patent['index'].tolist():
    df_i = df_lb.loc[df_lb['IPC小类'] == i]
    name_i = df_i['IPC小类名称'].tolist()[0]
    ls_name.append(name_i)
df_patent.loc[:,'patent_name'] = ls_name

In [ ]:
# 模拟预测

In [24]:
df_ent = pd.read_excel('E:\\project_data\\10kangbashi\\kbs_ind_rca_1.xlsx',dtype={'Id':'str','rca_num':'float'})
df_ent_best = df_ent.loc[df_ent['tag'] == 'best']
code_ls_1 = df_ent_best['Id'].tolist()
# df_net_new = pd.read_csv('E:\\project_data\\08laibing\\ind_connect.csv',dtype = {'Source':'str','Target':'str','Weight':'float'})
df_net_new = pd.read_csv('E:\\project_data\\03beidaihe\\data\\gephi_total.csv',dtype = {'Source':'str','Target':'str','Weight':'float'})
threshold = df_net_new['Weight'].quantile(0.3)
df_nodes = pd.read_csv('E:\\project_data\\03beidaihe\\data\\gephi_total_nodes.csv',dtype = 'str')
code_total_ls = df_nodes['Id'].tolist()
code_ls = [x for x in code_total_ls if x in code_ls_1]

In [32]:
threshold = df_net_new['Weight'].quantile(0.3)
threshold

0.676056

In [27]:
_district_industryCOs=code_ls
_simulate_list = []
_simulate_list.append(_district_industryCOs)
# 从一步开始向多步扩展
flag = 1
t = 1
ls_forcast = []
while flag == 1:
    _exist_list = []
    for k in range(0,len(_simulate_list)):
        _exist_list.extend(_simulate_list[k])
    c = 0
    _possible_list = []
    for j in range(len(_simulate_list[len(_simulate_list)-1])):
        _industryCO = _simulate_list[len(_simulate_list)-1][j]
        #if _industryCO in industryCOs_product:
        # 从每一个优势产业开始向外找，一步联系值超过的节点
        _df_industryCO_proximity = df_net_new.loc[((df_net_new['Source']==_industryCO) | (df_net_new['Target']==_industryCO))&(df_net_new['Weight']>=threshold)]
        if len(_df_industryCO_proximity)>0:
            _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
            _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

            for m in range(len(_industryCOs_1)):
                _industryCO_ = _industryCOs_1[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_a.append('上游')
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1

            for m in range(len(_industryCOs_2)):
                _industryCO_ = _industryCOs_2[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_a.append('下游')
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1
    if c == 0:
        flag=0
    else:
        flag = 1
        t = t + 1
        _possible_list = list(set(_possible_list))
        _simulate_list.append(_possible_list)

In [28]:
df_forcast=pd.DataFrame(ls_forcast,columns=['t','origin','destination','direction'])
df_forcast_ = df_forcast.loc[df_forcast['t'] <= 2]
df_forcast_

,t,origin,destination,direction
0,1,7241,6450,上游
1,1,7241,5143,上游
2,1,7241,7113,上游
3,1,7241,7223,上游
4,1,7241,8415,上游
...,...,...,...,...
661,2,4813,7690,下游
662,2,4813,6890,下游
663,2,8211,8131,上游
664,2,8211,5223,下游


In [29]:
df_forcast_.to_csv('E:\\project_data\\10kangbashi\\predict.csv',index = False)

In [30]:
ls_local = df_ent['Id'].tolist()

# 图谱里所有的行业节点
df_total_nodes = pd.read_csv('E:\\project_data\\10kangbashi\\gephi_total_nodes.csv',dtype = 'str')
# 优势行业打上best标签
df_total_nodes.loc[df_total_nodes['Id'].isin(code_ls),['tag_new']] = 'best'

df_best = df_forcast_.loc[df_forcast_['t'] == 1]
# df_sec = df_forcast_.loc[df_forcast_['t'] == 2]
# df_thrd = df_forcast_.loc[df_forcast_['t'] == 3]

trans_frt = df_best['destination'].unique().tolist()
# trans_sec = df_sec['destination'].unique().tolist()
# trans_thrd = df_thrd['destination'].unique().tolist()

trans_frt_1 = [x for x in trans_frt if x not in ls_local]
trans_frt_2 = [x for x in trans_frt if x in ls_local]
# trans_sec_1 = [x for x in trans_sec if x not in ls_local]
# trans_sec_2 = [x for x in trans_sec if x in ls_local]
# trans_thrd_1 = [x for x in trans_thrd if x not in ls_local]
# trans_thrd_2 = [x for x in trans_thrd if x in ls_local]

df_total_nodes.loc[df_total_nodes['Id'].isin(trans_frt_1),['tag_new']] = 'trans_frt'
df_total_nodes.loc[df_total_nodes['Id'].isin(trans_frt_2),['tag_new']] = 'local_frt'
# df_total_nodes.loc[df_total_nodes['Id'].isin(trans_sec_1),['tag_new']] = 'trans_sec'
# df_total_nodes.loc[df_total_nodes['Id'].isin(trans_sec_2),['tag_new']] = 'local_sec'
# df_nodes.loc[df_nodes['Id'].isin(trans_thrd_1),['tag_new']] = 'trans_thrd'
# df_nodes.loc[df_nodes['Id'].isin(trans_thrd_2),['tag_new']] = 'local_thrd'

df_none = df_total_nodes.loc[df_total_nodes['tag_new'].isnull()]
ls_none = df_none['Id'].unique().tolist()
ls_local_normal = [x for x in ls_none if x in ls_local]
ls_none_normal = [x for x in ls_none if x not in ls_local]
df_total_nodes.loc[df_total_nodes['Id'].isin(ls_local_normal),['tag_new']] = 'local_normal'
df_total_nodes.loc[df_total_nodes['Id'].isin(ls_none_normal),['tag_new']] = 'none'

In [31]:
date = datetime.datetime.now().strftime('%Y-%m-%d')
df_total_nodes.to_csv('E:\\project_data\\10kangbashi\\tupu_predict_'+date+'.csv',index = False,mode = 'w')

In [ ]:
# 模拟结束

In [79]:
df_none = df_nodes.loc[df_nodes['tag_new'].isnull()]

In [80]:
df_none

,Id,Label,tag_new
0,1361,水产品冷冻加工,NaN
1,1369,其他水产品加工,NaN
2,2442,体育器材及配件制造,NaN
3,4119,其他日用杂品制造,NaN
4,1783,纺织带和帘子布制造,NaN
...,...,...,...
599,4027,核子及核辐射测量仪器制造,NaN
600,4042,眼镜制造,NaN
601,4111,鬃毛加工、制刷及清扫工具制造,NaN
602,4130,核辐射加工,NaN


In [78]:
df_nodes

,Id,Label,tag_new
0,1361,水产品冷冻加工,NaN
1,1369,其他水产品加工,NaN
2,2442,体育器材及配件制造,NaN
3,4119,其他日用杂品制造,NaN
4,1783,纺织带和帘子布制造,NaN
...,...,...,...
599,4027,核子及核辐射测量仪器制造,NaN
600,4042,眼镜制造,NaN
601,4111,鬃毛加工、制刷及清扫工具制造,NaN
602,4130,核辐射加工,NaN


In [23]:
df_nodes

,Id,tag
0,1099,best
1,159,best
2,313,best
3,2812,best
4,162,best
...,...,...
65,3312,local_sec
66,1462,local_sec
67,1310,trans_sec
68,3515,local_sec


In [37]:
df_result

,Id,tag_x,Label,count_laibing,count_laibing_total,industry_code,count,count_total,rca_num,I,label,tag_y
0,1099,best,其他未列明非金属矿采选,47.0,32273.0,B1099,6249.0,39645329.0,9.239320,B,label_1,1.0
1,159,best,其他水果种植,705.0,32273.0,A0159,140062.0,39645329.0,6.183317,A,label_1,1.0
2,2012,best,木片加工,75.0,32273.0,C2012,17295.0,39645329.0,5.327131,C,label_1,1.0
3,313,best,猪的饲养,908.0,32273.0,A0313,219948.0,39645329.0,5.071291,A,label_1,1.0
4,2812,best,人造纤维（纤维素纤维）制造,6.0,32273.0,C2812,1650.0,39645329.0,4.467042,C,label_1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
812,3393,local_normal,锻件及粉末冶金制品制造,1.0,32273.0,C3393,18727.0,39645329.0,0.065597,C,label_4,NaN
813,1922,local_normal,皮箱、包(袋)制造,1.0,32273.0,C1922,26947.0,39645329.0,0.045587,C,label_4,NaN
814,3329,local_normal,其他金属工具制造,1.0,32273.0,C3329,27243.0,39645329.0,0.045092,C,label_4,NaN
815,4390,local_normal,其他机械和设备修理业,1.0,32273.0,C4390,33268.0,39645329.0,0.036925,C,label_4,NaN


In [36]:
ls_local = df_ent['Id'].tolist()
ls1 = df_forcast_2['origin'].unique().tolist()
ls2 = df_forcast_2['destination'].unique().tolist()
ls = ls1+ls2
ls = list(Counter(ls))
df_nodes = pd.DataFrame(ls,columns = ['Id'])
df_best = df_forcast_2.loc[df_forcast_2['t'] == 1]
ls_best = df_best['origin'].unique().tolist()
trans_frt = df_best['destination'].unique().tolist()
trans_frt_1 = [x for x in trans_frt if x not in ls_local]
trans_frt_2 = [x for x in trans_frt if x in ls_local]
trans_sec = df_forcast_2.loc[df_forcast_2['t'] == 2]['destination'].unique().tolist()
trans_sec_1 = [x for x in trans_sec if x not in ls_local]
trans_sec_2 = [x for x in trans_sec if x in ls_local]
df_nodes.loc[df_nodes['Id'].isin(ls_best),['tag']] = 'best'
df_nodes.loc[df_nodes['Id'].isin(trans_frt_1),['tag']] = 'trans_frt'
df_nodes.loc[df_nodes['Id'].isin(trans_frt_2),['tag']] = 'local_frt'
df_nodes.loc[df_nodes['Id'].isin(trans_sec_1),['tag']] = 'trans_sec'
df_nodes.loc[df_nodes['Id'].isin(trans_sec_2),['tag']] = 'local_sec'
df_result = pd.merge(left = df_nodes, right = df_ent,on = ['Id'],how = 'outer')
df_result.loc[df_result['tag_x'].isnull(),['tag_x']] = 'local_normal'

In [38]:
date = datetime.datetime.now().strftime('%Y-%m-%d')
df_result.to_csv('E:\\project_data\\08laibing\\ind_predict_erchan_1'+date+'.csv',index = False,mode = 'w')

In [20]:
df_data_total = df_total_new.loc[:,['industry_code','count']]
df_count_total = df_data_total.groupby(by=['industry_code'])['count'].sum().reset_index()
df_count_total_laibing = df_count_total.loc[df_count_total['industry_code'].isin(df_rca['Id'].tolist())]
df_count_total_laibing

,industry_code,count
0,A0111,202532
2,A0113,73376
3,A0119,125787
4,A0121,14627
5,A0122,20811
...,...,...
1339,R9051,34065
1340,R9052,1240
1341,R9053,3436
1343,R9059,6537


In [21]:
df_rca

,Id,count_laibing,count_laibing_total,count_total
0,F5165,1401,32273,39645329
1,A0133,1047,32273,39645329
2,F5299,1030,32273,39645329
3,A0313,908,32273,39645329
4,L7212,791,32273,39645329
...,...,...,...,...
791,D4690,1,32273,39645329
792,I6520,1,32273,39645329
793,C2631,1,32273,39645329
794,I6512,1,32273,39645329


In [8]:
# 读取地区编码t_admin_division_code.csv
df_dis = pd.read_csv('E:\\project_tupu\\t_admin_division_code.csv',dtype ='str')
df_dis_ls=df_dis['type_code'].tolist()
df_dis

,type_code,admin_name,short_name,zip_code,citycode
0,100000,中国,中国,NaN,NaN
1,110000,北京,北京,NaN,NaN
2,110100,北京市,北京,100000,010
3,110101,东城区,东城,100000,010
4,110102,西城区,西城,100000,010
...,...,...,...,...,...
4314,820200,氹仔岛,氹仔岛,999078,00853
4315,820201,嘉模堂区,嘉模堂区,999078,00853
4316,820300,路环岛,路环岛,999078,00853
4317,820301,圣方济各堂区,圣方济各堂区,999078,00853


In [67]:
df_data_local = pd.read_csv('E:\\project_data\\07jinan_lixiaqu\\济南市企业_行业_产品.csv',dtype ='str')
df_data_local['ind']=df_data_local['compindu'].str[1:]
df_data_local

,eid,compname,compindu,item,industry_code,district_code,industry,district,ind
0,ff757ca9-8c69-4ef1-93b2-702f26b43664,济南方圣混凝土有限公司,C3072,砖,C30,370103,非金属矿物制品业,市中区,3072
1,ff757ca9-8c69-4ef1-93b2-702f26b43664,济南方圣混凝土有限公司,C3072,水泥,C30,370103,非金属矿物制品业,市中区,3072
2,ae9ed57b-ebe1-4ce1-a3d0-532f4a60af67,济南宝国信息科技有限公司,I6519,技术服务,I65,370112,软件和信息技术服务业,历城区,6519
3,4454d1cf-8157-4de5-9963-3a77182dce8c,豪宸（济南）商贸有限公司,F5127,会议及展览服务,F51,370124,批发业,平阴县,5127
4,5b966b4a-5fce-4697-a7e6-ef81be743b3c,济南市鸿旭明彩钢瓦有限公司,C3200,有色金属合金,C32,370114,有色金属冶炼和压延加工业,章丘区,3200
...,...,...,...,...,...,...,...,...,...
802498,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,专用设备,E49,370113,建筑安装业,长清区,4910
802499,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,系统集成,E49,370113,建筑安装业,长清区,4910
802500,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,通用设备,E49,370113,建筑安装业,长清区,4910
802501,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,泵,E49,370113,建筑安装业,长清区,4910


In [133]:
df_data_total_ = df_data_local.drop_duplicates(subset=['eid'])

In [152]:
# 读取2017年的济南市企业数据
df_2017 = pd.read_excel('E:\\project_data\\07jinan_lixiaqu\\结果汇总_一对多_2017存续企业.xlsx',dtype = 'str')
df_2017

,eid,compname,compindu,item,industry_code,district_code,industry,district,credit_no
0,ff757ca9-8c69-4ef1-93b2-702f26b43664,济南方圣混凝土有限公司,C3072,砖,C30,370103,非金属矿物制品业,市中区,913701037402220982
1,ff757ca9-8c69-4ef1-93b2-702f26b43664,济南方圣混凝土有限公司,C3072,水泥,C30,370103,非金属矿物制品业,市中区,913701037402220982
2,ae9ed57b-ebe1-4ce1-a3d0-532f4a60af67,济南宝国信息科技有限公司,I6519,技术服务,I65,370112,软件和信息技术服务业,历城区,91370112MA3EKBNT83
3,d25300f0-d847-4dad-bc45-65602824e5d2,山东澳宝商业管理有限公司,L7299,房地产中介服务,L72,370105,商务服务业,天桥区,91370105MA3CA24763
4,c58bb18f-267d-4762-aa53-7fabaa979e69,济南辉忠商贸有限公司,F5174,家具,F51,370105,批发业,天桥区,91370105MA3C6KFM6U
...,...,...,...,...,...,...,...,...,...
288004,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,专用设备,E49,370113,建筑安装业,长清区,91370481MA3CFFY96Y
288005,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,系统集成,E49,370113,建筑安装业,长清区,91370481MA3CFFY96Y
288006,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,通用设备,E49,370113,建筑安装业,长清区,91370481MA3CFFY96Y
288007,121959eb-c770-42e9-b435-080f7786e0c8,山东得辰建设工程有限公司,E4910,泵,E49,370113,建筑安装业,长清区,91370481MA3CFFY96Y


In [157]:

df_2017['ind']=df_2017['compindu'].str[1:]
df_data_total_ = df_2017.drop_duplicates(subset=['eid'])

In [159]:
# df_data_local.groupby(['ind','district_code'])
# 筛选小类行业和对应地区的数据集
ls_ind = df_ind['ind4'].unique().tolist()
ls_dis = [x for x in df_data_total_['district_code'].unique().tolist() if type(x) != float]
df_data_local_new = df_data_total_.loc[(df_data_total_['ind'].isin(ls_ind))&(df_data_total_['district_code'].isin(ls_dis))]

# 把数据集进一步处理出来
df_data_local_new_ = df_data_local_new.loc[:,['ind','district_code']]
# 统计各地区各行业企业数量
df_jinan = df_data_local_new.value_counts(['ind','district_code']).reset_index()
df_lixia = df_jinan.loc[df_jinan['district_code'] == '370102']

# 得到历下区各行业企业数量和企业总数
lixia_total=df_lixia[0].sum()
df_lixia.loc[:,['lixia_total']] = lixia_total
df_lixia = df_lixia.rename(columns={0:'lixia'})
# 得到济南市各行业企业数量和企业总数
df_jinan_num = df_jinan.groupby('ind')[0].sum().reset_index().rename(columns={0:'num_jinan'})
jinan_total = df_jinan_num['num_jinan'].sum()
df_jinan_num.loc[:,['jinan_total']] = jinan_total

# 计算历下区各行业在济南市的优势值rca
df_result = pd.merge(df_lixia, df_jinan_num, left_on='ind', right_on='ind',how='inner')
df_result['rca_num']=(df_result['lixia']/df_result['lixia_total'])/(df_result['num_jinan']/df_result['jinan_total'])

# 找到符合要求的rca值用于判断优势产业
# _quantile = df_result['rca_num'].quantile(0.65)
# _quantile

C:\Users\duanyousheng\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [161]:
df_result.to_csv('E:\\project_data\\07jinan_lixiaqu\\jinan_2017.csv',index = False)

In [147]:

_quantile = df_lixia['lixia'].quantile(0.6)
_quantile

7.0

In [151]:
len(df_result.loc[(df_result['lixia'] == 1)&(df_result['num_jinan'] == 1)])

18

In [145]:
df_result

,ind,district_code,lixia,lixia_total,num_jinan,jinan_total,rca_num
0,5165,370102,3557,45830,24640,249136,0.784747
1,6599,370102,3393,45830,10512,249136,1.754628
2,7590,370102,2993,45830,14306,249136,1.137300
3,6519,370102,2880,45830,10859,249136,1.441748
4,7299,370102,2059,45830,8357,249136,1.339345
...,...,...,...,...,...,...,...
632,3973,370102,1,45830,16,249136,0.339756
633,3854,370102,1,45830,9,249136,0.604010
634,3912,370102,1,45830,11,249136,0.494190
635,3891,370102,1,45830,15,249136,0.362406


In [149]:
df_lixia.loc[df_lixia['lixia']<=10]

,ind,district_code,lixia,lixia_total
2072,5125,370102,10,45830
2084,5115,370102,10,45830
2096,3424,370102,10,45830
2099,2438,370102,10,45830
2103,3525,370102,10,45830
...,...,...,...,...
7051,3973,370102,1,45830
7059,3854,370102,1,45830
7066,3912,370102,1,45830
7070,3891,370102,1,45830


In [136]:
df_result.to_csv('E:\\project_data\\07jinan_lixiaqu\\lixia_rca_new.csv',index = False)

In [131]:
df_result.to_csv('E:\\project_data\\07jinan_lixiaqu\\lixia_rca.csv',index = False)

In [ ]:
df_result.loc[df_result['']]

In [119]:
df_jinan_num

,ind,num_jinan,jinan_total
0,0111,115,672424
1,0112,348,672424
2,0113,72,672424
3,0119,475,672424
4,0121,13,672424
...,...,...,...
1003,9490,2,672424
1004,9519,1,672424
1005,9529,6,672424
1006,9610,4,672424


In [118]:
df_lixia

,ind,district_code,lixia,lixia_total
2,5165,370102,14208,116659
4,7590,370102,10195,116659
11,6599,370102,6762,116659
12,6519,370102,6599,116659
20,7299,370102,4188,116659
...,...,...,...,...
7034,3596,370102,1,116659
7042,3585,370102,1,116659
7044,3735,370102,1,116659
7056,3822,370102,1,116659


In [102]:
lixia_total=df_lixia[0].sum()
df_lixia.loc[:,['lixia_total']]=lixia_total


In [112]:
df_jinan.groupby('ind')[0].sum().reset_index().rename(columns={0:'num_jinan'})

,ind,num_jinan
0,0111,115
1,0112,348
2,0113,72
3,0119,475
4,0121,13
...,...,...
1003,9490,2
1004,9519,1
1005,9529,6
1006,9610,4


In [98]:
len(df_lixia['ind'].unique().tolist())

637

In [97]:
len(df_jinan['ind'].unique().tolist())

1008

In [17]:
df_data_local.loc[df_data_local['district_code'] == '370102'].to_csv('E:\\project_data\\07jinan_lixiaqu\\lixiaqu.csv',index = False)

In [38]:

df_data_local.loc[df_data_local['compname'].str.contains('浪潮软件科技有限公司')]

,eid,compname,compindu,item,industry_code,district_code,industry,district
406692,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,教育咨询服务,I65,370102,软件和信息技术服务业,历下区
406693,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,物业管理服务,I65,370102,软件和信息技术服务业,历下区
406694,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,技术转让服务,I65,370102,软件和信息技术服务业,历下区
406695,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,会议及展览服务,I65,370102,软件和信息技术服务业,历下区
406696,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,系统集成,I65,370102,软件和信息技术服务业,历下区
406697,db86d9f0-2b49-4151-bb3c-275a92fe2530,浪潮软件科技有限公司,I6519,技术服务,I65,370102,软件和信息技术服务业,历下区


In [153]:
# 根据产品编码找到该产品t步以内的所有连接节点
def find_connect_nodes(code,t,df_net):
    # code是节点编码，str
    # df_nodes是节点表，dataframe
    # df_net是联系表，dataframe
    code_ls = []
    code_ls.append([code])
    
    flag = 1
    step = 1
    ls_forcast = []
    while step <= t:
        _exist_list = []
        for k in range(len(code_ls)):
            _exist_list.extend(code_ls[k])
        _possible_list = []
        
        for i in range(len(code_ls[-1])):
            _industryCO = code_ls[-1][i]
            df_temp = df_net.loc[(df_net['source'] == _industryCO) | (df_net['target'] == _industryCO)]
            if len(df_temp) > 0:
                _industryCOs_1 = df_temp.loc[:,'source'].unique()
                _industryCOs_2 = df_temp.loc[:,'target'].unique()
                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if _industryCO_ not in _exist_list:
                        _possible_list.append(_industryCO_)
                        ls_forcast.append(_industryCO_)

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if _industryCO_ not in _exist_list:
                        _possible_list.append(_industryCO_)
                        ls_forcast.append(_industryCO_)

        _possible_list = list(set(_possible_list))
        code_ls.append(_possible_list)
        step += 1
    return list(set(ls_forcast))

# 根据产品名称找到对应的产品编码
def name_to_code(name_ls,dataframe):
    # name_ls是要找的产品名称列表，list
    # dataframe是入库表
    code_ls=[]
    for a in name_ls:
        location = np.where(dataframe == a)
        columns_ls=dataframe.columns.tolist()
        
        
        for i in range(len(location[0])):
            index=location[0][i]
            column=location[1][i]
            if column < 8:
                continue
            else:
                tier=columns_ls[column][-1]
                col_name='code_'+tier
                code_ls.append(dataframe.iloc[index][col_name])
                
    return list(Counter(code_ls))

# 根据产品名称找到对应的产品编码
def name_to_code_dict(name_ls,dataframe):
    # name_ls是要找的产品名称列表，list
    # dataframe是c产品字典
    code_ls=[]
    for a in name_ls:
        df_a = dataframe.loc[dataframe['Label'] == a]
        code_a = df_a['Id'].unique().tolist()
        if len(code_a) == 0:
            print('bug',a)
            return code_ls
        else:
            code_ls.extend(code_a)
    return code_ls

# 根据产品编码找到产品名称
def code_to_name_dict(code_ls,dataframe):
    name_ls = []
    for a in code_ls:
        df_a = dataframe.loc[dataframe['Id'] == a]
        name_a = df_a['Label'].unique().tolist()
        if len(name_a) == 0:
            print('bug',a)
            return name_ls
        else:
            name_ls.extend(name_a)
    return name_ls

In [3]:
df_net = pd.read_excel('E:\\project_tupu\\chain\\yulin_total_connection_new.xlsx',dtype = {'source':'str','target':'str','weight':'float'})
df_net_new = pd.read_csv('E:\\project_tupu\\chain\\yulin_total_connection_.csv',dtype = {'source':'str','target':'str','weight':'float'},encoding = 'gbk')
df_nodes_dict = pd.read_csv('E:\\project_tupu\\chain\\dict_product.csv',dtype = 'str',encoding = 'gbk')
df_nodes_dataset = pd.read_csv('E:\\project_tupu\\chain\\yulin_dataset.csv',dtype = 'str',encoding='gbk')

In [7]:
df_net = pd.read_excel('E:\\project_tupu\\chain\\yulin_total_connection_.xlsx',dtype = {'source':'str','target':'str','weight':'float'})
df_net_new = pd.read_excel('E:\\project_tupu\\chain\\res_searc.xlsx',dtype = {'source':'str','target':'str','primary_weight':'float'})
df_nodes_dict = pd.read_excel('E:\\project_tupu\\chain\\dict_product.xlsx',dtype = 'str')
df_nodes_dataset = pd.read_csv('E:\\project_tupu\\chain\\yulin_dataset.csv',dtype = 'str',encoding='gbk')

In [39]:
# 浪潮
df_ent = df_data_local.loc[df_data_local['compname'].str.contains('浪潮软件科技有限公司')]
pro_ls = df_ent['item'].unique().tolist()
code_ls = name_to_code_dict(pro_ls,df_nodes_dict)

In [14]:
code_ls = ['7407030000',
'7403',
'7402',
'7407010000',
'69',
'7402050000',
'6904',
'690301',
'690302',
'6905020000',
'6904030000',
'6905010000',
'6903010100',
'6905',
'6903']

In [11]:
code_ls = ['7408010000', '7202', '7702040000', '7415', '610102', '7702']

In [118]:
df_net_new

,id,source,source_name,primary_type,target,target_name,related_type,relationship,primary_weight,related_weight,graph_type,proportion,del_flag,create_id,create_time,update_id,update_time
0,1.0,3130710,钢材,M,36010101,钻井机,NaN,0.0,2.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
1,2.0,3130710,钢材,M,3512010,石油钻井设备,NaN,0.0,2.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
2,3.0,3130711,铁道用钢材,M,36010101,钻井机,NaN,0.0,1.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
3,4.0,3130711,铁道用钢材,M,3512010,石油钻井设备,NaN,0.0,1.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
4,5.0,3130712,轻轨,M,36010101,钻井机,NaN,0.0,1.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37223,36729.0,1,农业产品,M,10201,马铃薯,NaN,0.0,3.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
37224,36739.0,50102,农产品初加工服务,T,1302012100,小米,NaN,0.0,3.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
37225,36861.0,40090104,系统形式自动数据处理设备,T,3749010,民用无人机,NaN,0.0,2.5,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
37226,36862.0,7609,工程设计服务,T,3749010,民用无人机,NaN,0.0,2.5,1.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN


In [27]:
# 奇安信
# 
code_ls_name = ['软件服务','广告服务','电子出版物出版服务','技术转让服务','安全软件产品']
code_ls = name_to_code_dict(code_ls_name,df_nodes_dict)
# code_ls_name_1 = ['互联网信息服务', '计算机系统服务', '技术服务', '存储器', '系统集成', '技术转让服务']
# code_ls = name_to_code_dict(code_ls_name_1,df_nodes_dict)


code_ls_name = ['软件技术服务','电子出版物出版服务','技术转让服务','安全软件产品','互联网信息服务', '计算机系统服务', '技术服务','系统集成']
code_ls = name_to_code_dict(code_ls_name,df_nodes_dict)

In [113]:
find_connect_nodes(code_ls[0],1,df_net_new)

['4009010299',
 '3908020200',
 '3922040',
 '3922050',
 '402105',
 '390802',
 '3973731',
 '3979030',
 '3594010',
 '40170304',
 '4017010203',
 '4009010204',
 '3973710',
 '760910',
 '3911040',
 '3632050500',
 '402106',
 '4011021100',
 '402102',
 '4017010101',
 '3911030',
 '3973730',
 '3982710',
 '402104',
 '3908020100',
 '3653010200',
 '4017010201',
 '7605140000',
 '4017010302',
 '4009010205',
 '402103',
 '402101',
 '3911010',
 '4009010203',
 '4009010102',
 '3972720',
 '3911050',
 '4017010102',
 '4017010202',
 '3911060',
 '4009010101',
 '4021990000',
 '3973732',
 '4021060200',
 '3911020',
 '4017010301']

In [172]:
# 来宾市
df_dianzi = pd.read_excel('E:\\project_data\\08laibing\\三条产业链主要企业.xlsx',sheet_name='电子信息产品烁姐',dtype = 'str')
df_yelian = pd.read_excel('E:\\project_data\\08laibing\\三条产业链主要企业.xlsx',sheet_name='Sheet9',dtype = 'str')
df_shipin = pd.read_excel('E:\\project_data\\08laibing\\三条产业链主要企业.xlsx',sheet_name='Sheet10',dtype = 'str')


In [174]:
# 电子信息
count = 1
code_ls = []
while count<=5:

    name_1 = 'code_'+str(count)
    product_ls = df_dianzi[name_1].unique().tolist()
    code_ls.extend(product_ls)
    print(name_1)
    count+=1
code_ls = [x for x in code_ls if x != '0']
code_ls = list(Counter(code_ls))

code_1
code_2
code_3
code_4
code_5


In [107]:
# 查看一共产业链关联了多少个产品
df_dianzi_chain = pd.read_csv('E:\\project_data\\08laibing\\df_dianzi_1.csv',dtype = {'t':'int','origin':'str','destination':'str'})
df_yelian_chain = pd.read_csv('E:\\project_data\\08laibing\\df_yelian_1.csv',dtype = {'t':'int','origin':'str','destination':'str'})
df_shipin_chain = pd.read_csv('E:\\project_data\\08laibing\\df_shipin_1.csv',dtype = {'t':'int','origin':'str','destination':'str'},encoding='gbk')

In [108]:
df_dianzi_chain_3 =  df_dianzi_chain.loc[df_dianzi_chain['t'] <= 3]
df_yelian_chain_2 = df_yelian_chain.loc[df_yelian_chain['t'] <= 2]
df_shipin_chain_3 = df_shipin_chain.loc[df_shipin_chain['t'] <= 3]

In [111]:
pd.DataFrame(ls_t).to_csv('E:\\project_data\\08laibing\\dianzi_chain_1.csv',index = False)

In [109]:
# 电子信息
ls1 = df_dianzi_chain_3['origin'].unique().tolist()
ls2 = df_dianzi_chain_3['destination'].unique().tolist()
ls_t = ls1 + ls2
ls_t = list(Counter(ls_t))

In [89]:
# 冶炼
ls1 = df_yelian_chain_2['origin'].unique().tolist()
ls2 = df_yelian_chain_2['destination'].unique().tolist()
ls_t = ls1 + ls2
ls_t = list(Counter(ls_t))

In [98]:
# 食品
ls1 = df_shipin_chain_3['origin'].unique().tolist()
ls2 = df_shipin_chain_3['destination'].unique().tolist()
ls_t = ls1 + ls2
ls_t = list(Counter(ls_t))

In [110]:
len(ls_t)

710

In [200]:
len(code_ls)

566

In [199]:
# 莆田
# 读取鞋服产业链的所有产品
df_xiefu = pd.read_excel('E:\\project_data\\09putian\\鞋服产业链 - 副本.xlsx',dtype = 'str',sheet_name = '鞋服制造')

count = 1
code_ls = []
while count<=5:
    index_1 = count
    index_2 = count+2
    name_1 = 'code_'+str(count)
    product_ls = df_xiefu[name_1].unique().tolist()
    code_ls.extend(product_ls)
    print(name_1)
    count+=1
code_ls = [x for x in code_ls if x != '0']
code_ls = list(Counter(code_ls))
# code_ls = ['4011020800']

code_1
code_2
code_3
code_4
code_5


In [201]:
_district_industryCOs=code_ls
_simulate_list = []
_simulate_list.append(_district_industryCOs)
# 从一步开始向多步扩展
flag = 1
t = 1
ls_forcast = []
while flag == 1:
    _exist_list = []
    for k in range(0,len(_simulate_list)):
        _exist_list.extend(_simulate_list[k])
    c = 0
    _possible_list = []
    for j in range(len(_simulate_list[len(_simulate_list)-1])):
        _industryCO = _simulate_list[len(_simulate_list)-1][j]
        #if _industryCO in industryCOs_product:
        # 从每一个优势产业开始向外找，一步联系值超过的节点
        _df_industryCO_proximity = df_net_new.loc[((df_net_new['source']==_industryCO) | (df_net_new['target']==_industryCO))&(df_net_new['primary_weight']>=1)]
        if len(_df_industryCO_proximity)>0:
            _industryCOs_1 = _df_industryCO_proximity.loc[:,'source'].unique()
            _industryCOs_2 = _df_industryCO_proximity.loc[:,'target'].unique()

            for m in range(len(_industryCOs_1)):
                _industryCO_ = _industryCOs_1[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_a.append('上游')
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1

            for m in range(len(_industryCOs_2)):
                _industryCO_ = _industryCOs_2[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_a.append('下游')
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1
    if c == 0:
        flag=0
    else:
        flag = 1
        t = t + 1
        _possible_list = list(set(_possible_list))
        _simulate_list.append(_possible_list)

In [202]:
len(ls_forcast)

25491

In [190]:
ls_forcast

[[1, '1800010', '2812032', '上游'],
 [1, '1800010', '2802020301', '上游'],
 [1, '1800010', '2802020302', '上游'],
 [1, '1800010', '2802020399', '上游'],
 [1, '1800010', '1701020301', '上游'],
 [1, '1800010', '17020102', '上游'],
 [1, '1800010', '2653010', '上游'],
 [1, '1800010', '2653060', '上游'],
 [1, '1800010', '2820010', '上游'],
 [1, '1800010', '2803020299', '上游'],
 [1, '1800010', '28030502', '上游'],
 [1, '1800010', '1701020303', '上游'],
 [1, '1800010', '1702010299', '上游'],
 [1, '1800010', '2828000', '上游'],
 [1, '1800010', '2829000', '上游'],
 [1, '1800010', '2803990000', '上游'],
 [1, '1800010', '280309', '上游'],
 [1, '1800010', '3551010', '上游'],
 [1, '1800010', '36300102', '上游'],
 [1, '1800010', '7611030000', '上游'],
 [1, '1800010', '1712010', '上游'],
 [1, '1800010', '1712050', '上游'],
 [1, '1800010', '1701050301', '上游'],
 [1, '1800010', '1701050302', '上游'],
 [1, '1800010', '1701050303', '上游'],
 [1, '1800010', '1701050304', '上游'],
 [1, '1800010', '1742020', '上游'],
 [1, '1800010', '1704020101', '上游'],
 [1,

In [192]:
df_forcast=pd.DataFrame(ls_forcast,columns=['t','origin','destination','direction'])
ls_ori = code_to_name_dict(df_forcast['origin'].tolist(),df_nodes_dict)
ls_des = code_to_name_dict(df_forcast['destination'].tolist(),df_nodes_dict)

df_forcast['ori_name'] = ls_ori
df_forcast['des_name'] = ls_des
df_forcast.to_csv('E:\\project_data\\09putian\\xiefu_t_1.csv',index = False, mode = 'w')

In [194]:
# 统计词频
df_v = pd.DataFrame(df_forcast['des_name'].tolist(),columns=['Id'])
df_frequency = df_v['Id'].value_counts().reset_index().rename(columns = {'index':'Id','Id':'frequency'})
print(df_frequency)
df_frequency.to_csv('E:\\project_data\\09putian\\xiefu_t_1_term_frequency.csv',index = False, mode = 'w')

In [196]:
# 下一步code_ls确定
df_t2 = pd.read_csv('E:\\project_data\\09putian\\xiefu_t_1_term_frequency.csv',dtype = 'str')
code_ls = df_t2['Id'].tolist()
code_ls

['印染布',
 '服装批发服务',
 '服装专门零售服务',
 '其他橡胶制品',
 '绵羊轻革',
 '猪轻革',
 '马轻革',
 '山羊轻革',
 '爬行动物轻革',
 '其他轻革',
 '轻革',
 '牛轻革',
 '拉链',
 '工业用缝纫机',
 '纺织、服装、鞋帽批发服务',
 '百货零售服务',
 '塑料人造革',
 '家庭洗衣服务',
 '涤纶纤维',
 '洗衣服务',
 '涤纶毛条',
 '精梳羊绒纱线',
 '粗梳羊绒纱线',
 '针织机械',
 '聚酯',
 '维纶纤维',
 '扣类制品及其零件',
 '毛纱',
 '羊毛纱',
 '棉布',
 '棉混纺绒布',
 '聚氨酯干法人造革',
 '棉混纺平布',
 'PBT纤维',
 '棉混纺布',
 '其他棉混纺布',
 '纯毛绒线',
 '绒线（俗称毛线）',
 '聚氨酯合成革',
 '色织布（含牛仔布）',
 '针织、编织物',
 '三醋酸纤维长丝',
 '其他蚕丝及交织机织物',
 '涤纶短纤维',
 '绢丝及交织机织物',
 '合成纤维纱',
 '衬里织物',
 '经编织物',
 '合成纤维与人造纤维混纺纱',
 '锦纶纤维',
 '桑蚕丝及交织机织物',
 '纺织专用设备',
 '聚氨酯塑料',
 '其他毛纱',
 '生物基化学纤维',
 '合成纤维单体',
 '化学纤维短纤绒布',
 '丙纶纤维长丝',
 '二醋酸纤维长丝',
 '木材采伐产品',
 '纬编制物',
 '不锈钢厨房整体橱柜',
 '?丝及交织机织物',
 '化学纤维短纤混纺平布',
 '柞蚕丝及交织机织物',
 '船舶修理',
 '其他合成纤维毛条',
 '合成纤维毛条',
 '高性能化学纤维',
 '丁苯橡胶',
 '线',
 '时装设计服务',
 '纺织科学技术研究服务',
 '木质厨房整体橱柜',
 '聚氯乙烯人造革',
 '化学纤维短纤布',
 '化学纤维短纤混纺绒布',
 '针织钩编织物（针织坯布）',
 '绢纺丝',
 '化学纤维短纤平布',
 '蚕丝',
 '其他涤纶长丝',
 '布',
 '广告服务',
 '合成纤维聚合物',
 '家具',
 '其他醋酸纤维长丝丝束',
 '纱',
 '活鸭',
 '醋酸纤维长丝',
 '其他合成纤维',
 '毛机织物（呢绒）',
 '蚕丝及交织机织物（含蚕丝≥30

In [179]:
df_forcast=pd.DataFrame(ls_forcast,columns=['t','origin','destination','direction'])

df_forcast.to_csv('E:\\project_data\\08laibing\\dianzi_shuojie.csv',index = False, mode = 'w')

In [180]:
df_gephi_1 = df_forcast.loc[df_forcast['direction'] == '上游']
df_gephi_2 = df_forcast.loc[df_forcast['direction'] == '下游']
df_gephi_1 = df_gephi_1[['destination','origin','direction']]
df_gephi_1.rename(columns = {'destination':'source','origin':'target'},inplace= True)
df_gephi_2 = df_gephi_2[['origin','destination','direction']]
df_gephi_2.rename(columns = {'origin':'source','destination':'target'},inplace= True)
df_gephi = pd.concat([df_gephi_1,df_gephi_2],axis = 0)
df_gephi.to_csv('E:\\project_data\\08laibing\\dianzi_gephi.csv',index = False)

In [181]:
ls1 = df_gephi['source'].unique().tolist()
ls2 = df_gephi['target'].unique().tolist()
ls_total = ls1 + ls2
ls_total = list(Counter(ls_total))
len(ls_total)

3554

In [182]:
df_nodes = pd.DataFrame(ls_total,columns=['Id'])
ls_name = code_to_name_dict(ls_total,df_nodes_dict)
df_nodes['Label'] = ls_name
ls_ind = []

for a in ls_total:
    df_a = df_nodes_dict.loc[df_nodes_dict['Id'] == a]
    name_a = df_a['industry_code'].unique().tolist()
    if len(name_a) == 0:
        print('bug',a)
    else:
        ls_ind.extend(name_a)
df_nodes['ind_f'] = ls_ind
df_nodes['ind'] = df_nodes['ind_f'].apply(lambda x:x[0])
df_nodes

,Id,Label,ind_f,ind
0,300113,降解塑料制品,C2929,C
1,3252790,铝材,C3252,C
2,33360102,铝型材,C3252,C
3,3252791,航空航天铝材,C3252,C
4,3251710,铜材,C3251,C
...,...,...,...,...
3549,270710,生物制剂,C2761,C
3550,27011301,肝素类,C2710,C
3551,2615040105,胰酶,C2662,C
3552,26190401,滤料,C2666,C


In [183]:
df_nodes.to_csv('E:\\project_data\\08laibing\\dianzi_gephi_nodes.csv',index = False)

In [184]:
# 莆田统计鞋服产业链各产品词频
ls1 = df_gephi['source'].tolist()
ls2 = df_gephi['target'].tolist()
ls = ls1 + ls2
df_v = pd.DataFrame(ls,columns=['Id'])
df_frequency = df_v['Id'].value_counts().reset_index().rename(columns = {'index':'Id','Id':'frequency'})
name_fre_ls = code_to_name_dict(df_frequency['Id'].tolist(),df_nodes_dict)
df_frequency['name'] = name_fre_ls

In [185]:
df_frequency.to_csv('E:\\project_data\\08laibing\\dianzi_term_frequency.csv', index = False)

In [84]:
# 读取基础图谱文件subClass_PP_bisquare_0.4_2022_MST_0_0585.csv
df_basic = pd.read_csv('E:\\pingxiang\\subClass_PP_bisquare_0.4_2022_MST_0_0585_1.csv',dtype ={'Source':'str','Target':'str','Weight':'float','Type':'str'})
df_basic

,Source,Target,Weight,Type
0,1810,2231,0.717391,undirected
1,1361,1369,0.714286,undirected
2,1783,4119,0.711111,undirected
3,3952,3963,0.705882,undirected
4,3333,3422,0.701493,undirected
...,...,...,...,...
1630,5136,8930,0.596491,undirected
1631,6940,8941,0.630435,undirected
1632,6891,8942,0.610390,undirected
1633,5172,8949,0.446429,undirected


In [5]:
# 读取萍乡企业数据
df_pingxiang_basic = pd.read_excel('E:\\pingxiang\\pingxiang_20220228T130440.xlsx',dtype ={'industry_code':'str','num':'int'})
df_pingxiang_basic

,eid,id,name,reg_no,category,econ_kind,i,industry_code,regist_capi_new,currency_unit,...,total_percent,path,row_update_time,create_time,is_valid,longitude,latitude,gd_longitude,gd_latitude,credit_no
0,c58b16b3-927f-4315-b979-9eb9cf2a5150,554158,海安县弹力絮棉厂,13854369,E,NaN,R,8600,0.0,CNY,...,0.0,NaN,NaT,NaT,NaN,107.075527,27.221801,-1.000000,-1.000000,NaN
1,d52e34ae-61ef-4ad1-b508-c2cdf29d994a,712996,萍乡市安源区科学技术协会,NaN,E,事业单位,NaN,NaN,0.0,NaN,...,0.0,NaN,NaT,NaT,NaN,-1.000000,-1.000000,-1.000000,-1.000000,123603020674951008
2,3fa2e1ce-8b20-448e-99ba-f975fa1ff9bb,195816,萍乡市湘东区荷尧镇敬老院,未公示,E,事业单位,NaN,NaN,0.0,NaN,...,0.0,NaN,NaT,NaT,NaN,-1.000000,-1.000000,-1.000000,-1.000000,12360313060787204N
3,9042e322-8062-478d-a9ff-8f2f5dbfc9a3,416959,海安县青萍钱桥电器设备厂,30006,E,NaN,E,4900,0.0,CNY,...,0.0,NaN,NaT,NaT,NaN,-1.000000,-1.000000,-1.000000,-1.000000,NaN
4,b5d9f7a7-2352-42d7-a4cf-4af8e142b90f,579534,萍乡市社会保险事业管理局,NaN,E,事业单位,NaN,NaN,0.0,NaN,...,0.0,NaN,NaT,NaT,NaN,-1.000000,-1.000000,-1.000000,-1.000000,123603004913306000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,a8519462-89f8-47b6-9e6f-d7a947348210,543598,易政伟,360301600033893,P,个体工商户,C,2600,0.0,NaN,...,0.0,NaN,NaT,NaT,NaN,117.133137,36.694854,102.100404,36.502554,NaN
7996,67303af6-168f-40b9-837a-8a848d6aa10f,333575,萍乡市天防智能门业制造厂,360301310000758,E,普通合伙企业,C,4190,50.0,CNY,...,0.5,"{""nodes"":[{""eid"":""67303af6-168f-40b9-837a-8a84...",2020-12-04 00:56:41,2020-04-16 10:06:46,1.0,113.889195,27.660629,113.871650,27.644136,
7997,5993e5c2-c831-11e7-90c1-00163e1251d8,346170,萍乡睿能光电材料研究中心,110014,E,民办非企业单位,NaN,NaN,0.0,NaN,...,0.0,NaN,NaT,NaT,NaN,-1.000000,-1.000000,-1.000000,-1.000000,5236030056107320XD
7998,bf33b3b0-e1ac-4c06-a5e7-b58178a103f9,711208,萍乡辉宏钢材批发部,360301600033924,P,个体工商户,F,5100,15.0,CNY,...,0.0,NaN,NaT,NaT,NaN,113.847198,27.672893,113.841810,27.663385,NaN


In [6]:
df_pingxiang_basic_2=df_pingxiang_basic['industry_code'].value_counts().reset_index()
df_pingxiang_basic_2=df_pingxiang_basic_2.rename(columns={'index':'industry','industry_code':'num'})

In [7]:
df_pingxiang_basic_2

,industry,num
0,5200,2218
1,5100,528
2,6200,447
3,8000,287
4,8100,244
...,...,...
384,5134,1
385,5241,1
386,2911,1
387,200,1


In [2]:
# 读取全国企业数据
df_total = pd.read_csv('E:\\pingxiang\\total_ind202109.csv',dtype ={'district_code':'str','industry_code':'str','count(*) ':'int'})
df_total['ind']=df_total['industry_code'].str[1:]
df_total

,district_code,industry_code,count(*),ind
0,540230,N7690,1,7690
1,350302,C3554,1,3554
2,450329,B0931,1,0931
3,640402,B1011,1,1011
4,350504,C1783,2,1783
...,...,...,...,...
2144342,230803,I6531,18,6531
2144343,130181,C2829,2,2829
2144344,510302,F5227,18,5227
2144345,130200,C4011,1,4011


In [3]:
ls_ind=['C2825', 'C3415', 'C3833', 'C3832', 'C3714']
ls_loc=['360300', '360302', '360313', '360321', '360322', '360323']
df_temp=df_total.loc[(df_total['district_code'].isin(ls_loc)) & (df_total['industry_code'].isin(ls_ind))]
df_temp

,district_code,industry_code,count(*),ind
210927,360323,C3415,1,3415
412019,360313,C3714,1,3714
991304,360302,C3832,1,3832
1218769,360302,C2825,1,2825
1343982,360322,C3415,2,3415
1495222,360323,C3833,29,3833
1605010,360302,C3714,1,3714
1729705,360302,C3415,1,3415
1792608,360321,C3832,1,3832
2085264,360313,C3832,1,3832


In [3]:
df_total=df_total.drop(df_total.loc[df_total['ind']=='0000'].index)
df_total

,district_code,industry_code,count(*),ind
0,540230,N7690,1,7690
1,350302,C3554,1,3554
2,450329,B0931,1,0931
3,640402,B1011,1,1011
4,350504,C1783,2,1783
...,...,...,...,...
2144342,230803,I6531,18,6531
2144343,130181,C2829,2,2829
2144344,510302,F5227,18,5227
2144345,130200,C4011,1,4011


In [5]:
# 读取地区编码t_admin_division_code.csv
df_dis = pd.read_csv('E:\\pingxiang\\t_admin_division_code.csv',dtype ='str')
df_dis_ls=df_dis['type_code'].tolist()

In [6]:
df_total=df_total.loc[df_total['district_code'].isin(df_dis_ls)]
df_total=df_total.rename(columns={'industry_code':'industry','count(*) ':'num'})
df_total

,district_code,industry,num,ind
0,540230,N7690,1,7690
1,350302,C3554,1,3554
2,450329,B0931,1,0931
3,640402,B1011,1,1011
4,350504,C1783,2,1783
...,...,...,...,...
2144342,230803,I6531,18,6531
2144343,130181,C2829,2,2829
2144344,510302,F5227,18,5227
2144345,130200,C4011,1,4011


In [7]:
df_pingxiang=df_total.loc[(df_total['district_code']=='360300')|(df_total['district_code']=='360302')|
                          (df_total['district_code']=='360313')|(df_total['district_code']=='360321')|
                          (df_total['district_code']=='360322')|(df_total['district_code']=='360323')]
df_pingxiang

,district_code,industry,num,ind
909,360323,F5261,47,5261
948,360302,O8070,3,8070
963,360321,L7219,34,7219
974,360322,C1810,6,1810
1333,360302,C1369,1,1369
...,...,...,...,...
2138790,360321,L7111,7,7111
2139240,360302,C4210,7,4210
2142280,360321,F5151,1,5151
2142572,360321,C1300,8,1300


In [102]:
df_pingxiang=df_total.loc[(df_total['district_code']=='360300')|(df_total['district_code']=='360302')|
                          (df_total['district_code']=='360313')|(df_total['district_code']=='360321')|
                          (df_total['district_code']=='360322')|(df_total['district_code']=='360323')]
df_pingxiang

,district_code,industry_code,count(*)
909,360323,F5261,47
948,360302,O8070,3
963,360321,L7219,34
974,360322,C1810,6
1333,360302,C1369,1
...,...,...,...
2138790,360321,L7111,7
2139240,360302,C4210,7
2142280,360321,F5151,1
2142572,360321,C1300,8


In [8]:
df_pingxiang=df_pingxiang.dropna()
df_pingxiang

,district_code,industry,num,ind
909,360323,F5261,47,5261
948,360302,O8070,3,8070
963,360321,L7219,34,7219
974,360322,C1810,6,1810
1333,360302,C1369,1,1369
...,...,...,...,...
2138790,360321,L7111,7,7111
2139240,360302,C4210,7,4210
2142280,360321,F5151,1,5151
2142572,360321,C1300,8,1300


In [9]:
df_total_ind=df_total.iloc[:,[1,2,3]]
df_pingxiang_ind=df_pingxiang.iloc[:,[1,2,3]]

In [112]:
df_total_ind=df_total.iloc[:,[1,2,3]]
df_pingxiang_ind=df_pingxiang.iloc[:,[1,2,3]]
df_total_ind=df_total_ind.rename(columns={'industry_code':'industry','count(*) ':'num'})
df_pingxiang_ind=df_pingxiang_ind.rename(columns={'industry_code':'industry','count(*) ':'num'})

In [10]:
df_px=df_pingxiang_ind.groupby(['industry'])['num'].sum()
df_px=df_px.reset_index()
df_px

,industry,num
0,A0100,350
1,A0110,6
2,A0111,520
3,A0112,1
4,A0113,4
...,...,...
1098,R9051,24
1099,R9052,4
1100,R9053,6
1101,R9059,13


In [ ]:
df_ch=df_total_ind.groupby(['industry'])['num'].sum()
df_ch=df_ch.reset_index()
ch_total=df_ch['num'].sum()
df_ch['px_total']=ch_total
df_ch

In [123]:
df_px=df_pingxiang_ind.groupby(['industry'])['num'].sum()
df_px=df_px.reset_index()
df_px

,industry,num
0,A0000,13
1,A0100,350
2,A0110,6
3,A0111,520
4,A0112,1
...,...,...
1104,R9051,24
1105,R9052,4
1106,R9053,6
1107,R9059,13


In [11]:
df_px=df_px.drop_duplicates(subset=['industry','num'],keep='first')
df_px

,industry,num
0,A0100,350
1,A0110,6
2,A0111,520
3,A0112,1
4,A0113,4
...,...,...
1098,R9051,24
1099,R9052,4
1100,R9053,6
1101,R9059,13


In [125]:
df_px=df_px.reset_index()
px_total=df_px['num'].sum()
df_px['px_total']=px_total
df_px

,index,industry,num,px_total
0,0,A0000,13,55189
1,1,A0100,350,55189
2,2,A0110,6,55189
3,3,A0111,520,55189
4,4,A0112,1,55189
...,...,...,...,...
1104,1104,R9051,24,55189
1105,1105,R9052,4,55189
1106,1106,R9053,6,55189
1107,1107,R9059,13,55189


In [ ]:
53599984

In [105]:
df_ch=df_total_ind.groupby(['industry'])['num'].sum()
df_ch=df_ch.reset_index()
ch_total=df_ch['num'].sum()
df_ch['px_total']=ch_total
df_ch

,industry,num,px_total
0,A0000,18185,53600088
1,A0100,359530,53600088
2,A0110,57965,53600088
3,A0111,188447,53600088
4,A0112,137630,53600088
...,...,...,...
1755,S9542,1,53600088
1756,S9600,522,53600088
1757,S9610,91,53600088
1758,S9620,120,53600088


In [126]:
df_pingxiang_result=pd.merge(df_px, df_ch, left_on='industry', right_on='industry',how='inner')
df_pingxiang_result['ind']=df_pingxiang_result['industry'].str[1:]
df_pingxiang_result['ind']=df_pingxiang_result['ind'].astype('int')
df_pingxiang_result['ind']=df_pingxiang_result['ind'].astype('str')
df_pingxiang_result

,index,industry,num_x,px_total_x,num_y,px_total_y,ind
0,0,A0000,13,55189,18185,53600088,0
1,1,A0100,350,55189,359530,53600088,100
2,2,A0110,6,55189,57965,53600088,110
3,3,A0111,520,55189,188447,53600088,111
4,4,A0112,1,55189,137630,53600088,112
...,...,...,...,...,...,...,...
1104,1104,R9051,24,55189,32791,53600088,9051
1105,1105,R9052,4,55189,1153,53600088,9052
1106,1106,R9053,6,55189,3465,53600088,9053
1107,1107,R9059,13,55189,6421,53600088,9059


In [127]:
df_pingxiang_result['rca_dys']=(df_pingxiang_result['num_x']/df_pingxiang_result['px_total_x'])/(df_pingxiang_result['num_y']/df_pingxiang_result['px_total_y'])
df_pingxiang_result

,index,industry,num_x,px_total_x,num_y,px_total_y,ind,rca_dys
0,0,A0000,13,55189,18185,53600088,0,0.694293
1,1,A0100,350,55189,359530,53600088,100,0.945466
2,2,A0110,6,55189,57965,53600088,110,0.100531
3,3,A0111,520,55189,188447,53600088,111,2.679952
4,4,A0112,1,55189,137630,53600088,112,0.007057
...,...,...,...,...,...,...,...,...
1104,1104,R9051,24,55189,32791,53600088,9051,0.710836
1105,1105,R9052,4,55189,1153,53600088,9052,3.369331
1106,1106,R9053,6,55189,3465,53600088,9053,1.681748
1107,1107,R9059,13,55189,6421,53600088,9059,1.966318


In [128]:
df_pingxiang_result.to_csv('E:\\pingxiang\\pingxiang_dys_330_2.csv',index=False,mode='w')

In [85]:
df_pingxiang_result_rca_1=df_pingxiang_result.loc[df_pingxiang_result['rca_wmh']>=1]
df_pingxiang_result_rca_2=df_pingxiang_result.loc[df_pingxiang_result['rca_wmh']>=2]
df_pingxiang_result_rca_15=df_pingxiang_result.loc[df_pingxiang_result['rca_wmh']>=1.5]

In [86]:
industryCOs_product=df_pingxiang_result['ind'].tolist()
ls_best_1=df_pingxiang_result_rca_1['ind'].tolist()
ls_best_15=df_pingxiang_result_rca_15['ind'].tolist()
ls_best_2=df_pingxiang_result_rca_2['ind'].tolist()

In [89]:
ls_result_1=findpotential(ls_best_1,df_basic)
df_forcast_1=pd.DataFrame(ls_result_1,columns=['t','origin','destination'])
ls_result_15=findpotential(ls_best_15,df_basic)
df_forcast_15=pd.DataFrame(ls_result_15,columns=['t','origin','destination'])
ls_result_2=findpotential(ls_best_2,df_basic)
df_forcast_2=pd.DataFrame(ls_result_2,columns=['t','origin','destination'])

In [ ]:
df_basic=pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService_0.5128205128205128.csv',dtype ={'Source':'str','Target':'str','Weight':'float','Type':'str'})
industryCOs_product=df_result['industry'].tolist()
ls_result1=findpotential(ls_best,df_basic)
df_forcast=pd.DataFrame(ls_result1,columns=['t','origin','destination'])

In [142]:
df_basic_1=pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService_0.5128205128205128.csv',dtype ={'Source':'str','Target':'str','Weight':'float','Type':'str'})
df_pingxiang_result=pd.read_csv('E:\\pingxiang\\pingxiang_dys_330_2.csv')
industryCOs_product=df_pingxiang_result['ind_11'].tolist()

In [146]:
df_pingxiang_result=pd.read_csv('E:\\pingxiang\\pingxiang_dys_330_2.csv',dtype=str)
df_pingxiang_result_best=df_pingxiang_result.loc[df_pingxiang_result['rca_dys']>=2.5]
df_pingxiang_result_best=df_pingxiang_result_best.dropna()
ls_best_25=df_pingxiang_result_best['ind_11'].tolist()
industryCOs_product=ls_best_25
ls_result_25=findpotential(ls_best_25,df_basic_1)
df_forcast_25=pd.DataFrame(ls_result_1,columns=['t','origin','destination'])

In [148]:
df_forcast_25.to_csv('E:\\pingxiang\\pingxiang_dys_331_rca25_result.csv')

In [149]:
df_forcast_25

,t,origin,destination
0,1,111,412
1,1,1419,1411
2,1,1713,1783
3,1,1713,2231
4,1,1713,1810
...,...,...,...
250,2,3591,4014
251,2,7514,7512
252,3,3329,3479
253,3,3484,3459


In [153]:
df_pingxiang_result=pd.read_csv('E:\\pingxiang\\pingxiang_dys_330_2.csv')
df_pingxiang_result.loc[df_pingxiang_result['ind_11']==3473]

,industry,num_x,px_total_x,num_y,px_total_y,ind,rca_dys,ind_11
411,C3473,1,55189,736,53600088,3473,1.319578,3473.0


In [ ]:
ls_best_25

In [ ]:
_df_industryCO_proximity = df_proximity_productService.loc[((df_proximity_productService['Source']==_industryCO) | (df_proximity_productService['Target']==_industryCO))&(df_proximity_productService['Weight']>0.55)]

In [177]:
ls_result_25_1=findpotential1(ls_best_25,df_basic_1)
df_forcast_25_1=pd.DataFrame(ls_result_1,columns=['t','origin','destination'])
df_forcast_25_1.to_csv('E:\\pingxiang\\pingxiang_dys_330_3.csv')

In [181]:
_district_industryCOs=ls_best_25
_simulate_list = []
_simulate_list.append(_district_industryCOs)
# 从一步开始向多步扩展
flag = 1
t = 1
ls_forcast=[]
while flag==1:
    _exist_list = []
    for k in range(0,len(_simulate_list)):
        _exist_list.extend(_simulate_list[k])
    c = 0
    _possible_list = []
    for j in range(len(_simulate_list[len(_simulate_list)-1])):
        _industryCO = _simulate_list[len(_simulate_list)-1][j]
        #if _industryCO in industryCOs_product:
        # 从每一个优势产业开始向外找，一步联系值超过的节点
        _df_industryCO_proximity = df_basic_1.loc[((df_basic_1['Source']==_industryCO) | (df_basic_1['Target']==_industryCO))&(df_basic_1['Weight']>0.55)]
        if len(_df_industryCO_proximity)>0:
            _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
            _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

            for m in range(len(_industryCOs_1)):
                _industryCO_ = _industryCOs_1[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1

            for m in range(len(_industryCOs_2)):
                _industryCO_ = _industryCOs_2[m]
                if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                    _possible_list.append(_industryCO_)
                    ls_a=[]
                    ls_a.append(t)
                    ls_a.append(_industryCO)
                    ls_a.append(_industryCO_)
                    ls_forcast.append(ls_a)
                    #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                    #f.write(ss)
                    c= c+1
    if c == 0:
        flag=0
    else:
        flag = 1
        t = t + 1
        _possible_list = list(set(_possible_list))
        _simulate_list.append(_possible_list)

In [182]:
ls_forcast

[[1, '162', '8930'],
 [1, '162', '7213'],
 [1, '162', '6930'],
 [1, '390', '6020'],
 [1, '529', '6811'],
 [1, '529', '6820'],
 [1, '529', '6640'],
 [1, '529', '6990'],
 [1, '620', '6633'],
 [1, '810', '5990'],
 [1, '820', '8911'],
 [1, '820', '7213'],
 [1, '1011', '6020'],
 [1, '1011', '7282'],
 [1, '1099', '7483'],
 [1, '1493', '7231'],
 [1, '1493', '5162'],
 [1, '1493', '7450'],
 [1, '2041', '7213'],
 [1, '2041', '8930'],
 [1, '2423', '5189'],
 [1, '2423', '5810'],
 [1, '2423', '6790'],
 [1, '2512', '7450'],
 [1, '2671', '6930'],
 [1, '2671', '6920'],
 [1, '2825', '7213'],
 [1, '2915', '5139'],
 [1, '2915', '5189'],
 [1, '2915', '5134'],
 [1, '3071', '5131'],
 [1, '3120', '7482'],
 [1, '3120', '7231'],
 [1, '3120', '5164'],
 [1, '3562', '7211'],
 [1, '3610', '5125'],
 [1, '3610', '7330'],
 [1, '3620', '5121'],
 [1, '3620', '6713'],
 [1, '3832', '5139'],
 [1, '3832', '5132'],
 [1, '3832', '5139'],
 [1, '3832', '5132'],
 [1, '3841', '8930'],
 [1, '3953', '8610'],
 [1, '3969', '5133'],


In [184]:
df_forcast_25_2=pd.DataFrame(ls_forcast,columns=['t','origin','destination'])
df_forcast_25_2.to_csv('E:\\pingxiang\\pingxiang_dys_330_4.csv')

In [169]:
df_pingxiang_result=pd.read_csv('E:\\pingxiang\\pingxiang_dys_330_2.csv',dtype={'ind_11':'str'})
df_pingxiang_result_best=df_pingxiang_result.loc[df_pingxiang_result['rca_dys']>=2.5]
df_pingxiang_result_best=df_pingxiang_result_best.dropna()
ls_best_25=df_pingxiang_result_best['ind_11'].tolist()
f = open('E:\\pingxiang\\findPotential_2022.csv','w')
header = 't,origin,destination,prody\n'
f.write(header)

27

In [170]:
df_proximity_productService = pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService_0.5128205128205128.csv',dtype = {'Source':'str','Target':'str'})

df_type = pd.read_csv('E:\\pingxiang\\IndustryType3.csv')
df_type['Id'] = df_type['Id'].astype('str')
industryCOs_product = df_type[df_type['ProductServiceType']==2].loc[:,'Id'].unique()
industryCOs_service = df_type[df_type['ProductServiceType']==1].loc[:,'Id'].unique()
_district_industryCOs = ls_best_25
_simulate_list = []
_simulate_list.append(_district_industryCOs)
flag = 1
t = 1
while flag==1:
    
    _exist_list = []
    for k in range(0,len(_simulate_list)):
        _exist_list.extend(_simulate_list[k])
    c = 0
    _possible_list = []
    for j in range(len(_simulate_list[len(_simulate_list)-1])):
        _industryCO = _simulate_list[len(_simulate_list)-1][j]
        if _industryCO in industryCOs_product:
            _df_industryCO_proximity = df_proximity_productService.loc[((df_proximity_productService['Source']==_industryCO) | (df_proximity_productService['Target']==_industryCO))&(df_proximity_productService['Weight']>0.55)]
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        
                        ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        f.write(ss)
                        c= c+1

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        
                        ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        f.write(ss)
                        c= c+1
                        
        if _industryCO in industryCOs_service:
            df_industryCO_proximity = df_proximity_productService.loc[((df_proximity_productService['Source']==_industryCO) | (df_proximity_productService['Target']==_industryCO))&(df_proximity_productService['Weight']>0.55)]			
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()
                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ss =str(t) + ',' + _industryCO + ','+ _industryCO_ +'\n'
                        f.write(ss)
                        c= c+1
                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO):
                        _possible_list.append(_industryCO_)
                        ss =str(t) + ',' + _industryCO + ','+ _industryCO_ +'\n'
                        f.write(ss)
                        c= c+1
    if c==0:
        flag=0
    else:
        flag = 1
        t = t + 1
        _possible_list = list(set(_possible_list))
        _simulate_list.append(_possible_list)

In [2]:
df=pd.read_csv('E:\\pingxiang\\weightedExist_2022_23f3.csv',dtype ={'INDUSTRYCO_3':'str','NUM':'float'})
df=df[['INDUSTRYCO_3','NUM']]
df=df.groupby(['INDUSTRYCO_3'])['NUM'].sum()
df=df.reset_index()
df

,INDUSTRYCO_3,NUM
0,1011,20952.649019
1,1012,24780.793383
2,1013,1471.568093
3,1019,18304.646582
4,1020,1237.718915
...,...,...
1087,9430,3.331412
1088,9440,77.002827
1089,9510,292.381603
1090,9520,2166.193247


In [9]:
df=pd.read_csv('E:\\pingxiang\\weightedExist_2017_23f3.csv',dtype ={'INDUSTRYCO_3':'str','NUM':'float'})
df=df[['INDUSTRYCO_3','NUM_x']]
df=df.groupby(['INDUSTRYCO_3'])['NUM_x'].sum()
df=df.reset_index()
df

,INDUSTRYCO_3,NUM_x
0,1011,8314.0
1,1012,10039.0
2,1013,583.0
3,1019,7392.0
4,1020,462.0
5,1030,390.0
6,1091,74.0
7,1092,301.0
8,1093,168.0
9,1099,3650.0


In [3]:
df_basic = pd.read_csv('E:\\pingxiang\\subClass_PP_bisquare_0.4_2022_MST_0_0585_1.csv',dtype ={'Source':'str','Target':'str','Weight':'float','Type':'str'})
df_pingxiang = pd.read_csv('E:\\pingxiang\\11pingxiang_industry.csv',dtype ={'industry_code':'str','num':'float'})
df_industry = pd.read_csv('E:\\pingxiang\\pingxiang_total_industry.csv',dtype ={'INDUSTRYCO_3':'str','NUM':'float'})

In [37]:
df_industry=df

In [39]:
df_industry

,INDUSTRYCO_3,NUM
0,1011,20952.649019
1,1012,24780.793383
2,1013,1471.568093
3,1019,18304.646582
4,1020,1237.718915
...,...,...
1087,9430,3.331412
1088,9440,77.002827
1089,9510,292.381603
1090,9520,2166.193247


In [36]:
total_num=df_pingxiang_basic_2['num'].sum()
df_pingxiang_basic_2['total_num']=total_num
df_pingxiang_basic_2

,industry,num,total_num
0,5200,2218,6925
1,5100,528,6925
2,6200,447,6925
3,8000,287,6925
4,8100,244,6925
...,...,...,...
384,1439,1,6925
385,1440,1,6925
386,5242,1,6925
387,2661,1,6925


In [42]:
# 计算萍乡各产业的rca值
df_result=pd.merge(df_pingxiang_basic_2, df_industry, left_on='industry', right_on='INDUSTRYCO_3',how='inner')
#df_result=df_result.drop(columns=['Unnamed: 0'],axis=1)
df_result

,industry,num,total_num,INDUSTRYCO_3,NUM
0,5299,146,6925,5299,2.109622e+06
1,5165,90,6925,5165,2.259214e+06
2,5199,84,6925,5199,3.258277e+06
3,7010,60,6925,7010,9.842098e+05
4,5172,60,6925,5172,2.253511e+05
...,...,...,...,...,...
268,1439,1,6925,1439,5.741390e+03
269,1440,1,6925,1440,5.586737e+03
270,5242,1,6925,5242,3.212774e+04
271,2661,1,6925,2661,2.934509e+04


In [45]:
df_total_1=df_result.sum()
df_total_1
a=64148410.024613

In [46]:
df_result['total_num1']=a

df_result['rca_num']=(df_result['num']/df_result['total_num'])/(df_result['NUM']/df_result['total_num1'])
df_result.to_csv('E:\\pingxiang\\pingxiang_ind_rca_1.csv',index=False,mode='w')

In [49]:
df_industry_best=df_result.loc[df_result['rca_num']>1.5]
len(df_industry_best)

52

In [52]:
ls_best=df_industry_best['industry'].tolist()
len(ls_best)

52

In [54]:
df_basic=pd.read_csv('E:\\pingxiang\\addProducerService_weighted23f3_0.25_pp_2022_productService_MST_0.csv',dtype ={'Source':'str','Target':'str','Weight':'float','Type':'str'})
industryCOs_product=df_result['industry'].tolist()
ls_result1=findpotential(ls_best,df_basic)
df_forcast=pd.DataFrame(ls_result1,columns=['t','origin','destination'])
data_predict_new=df_forcast.loc[df_forcast['t']<=2]
df_forcast.to_csv('E:\\pingxiang\\11pingxiang_result0321_new_connection_1.csv',index=False,mode='w')

In [171]:
def findpotential1(_ls_predict_fir,df_product_net):
    # _district_industryCOs是优势节点和种子节点之和
    _district_industryCOs = _ls_predict_fir
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast=[]
    while flag==1:
        _exist_list = []
        for k in range(0,len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []
        for j in range(len(_simulate_list[len(_simulate_list)-1])):
            _industryCO = _simulate_list[len(_simulate_list)-1][j]
            #if _industryCO in industryCOs_product:
            # 从每一个优势产业开始向外找，一步联系值超过的节点
            _df_industryCO_proximity = df_product_net.loc[((df_product_net['Source']==_industryCO) | 
                                                           (df_product_net['Target']==_industryCO))&
                                                          (df_product_net['Weight']>0.55)]
            if len(_df_industryCO_proximity)>0:
                _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                for m in range(len(_industryCOs_1)):
                    _industryCO_ = _industryCOs_1[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1

                for m in range(len(_industryCOs_2)):
                    _industryCO_ = _industryCOs_2[m]
                    if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                        _possible_list.append(_industryCO_)
                        ls_a=[]
                        ls_a.append(t)
                        ls_a.append(_industryCO)
                        ls_a.append(_industryCO_)
                        ls_forcast.append(ls_a)
                        #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                        #f.write(ss)
                        c= c+1
        if c == 0:
            flag=0
        else:
            flag = 1
            t = t + 1
            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast

In [145]:
def findpotential1(_ls_predict_fir,df_product_net):
    # _district_industryCOs是优势节点和种子节点之和
    _district_industryCOs = _ls_predict_fir
    _simulate_list = []
    _simulate_list.append(_district_industryCOs)
    # 从一步开始向多步扩展
    flag = 1
    t = 1
    ls_forcast=[]
    while flag==1:
        _exist_list = []
        for k in range(0,len(_simulate_list)):
            _exist_list.extend(_simulate_list[k])
        c = 0
        _possible_list = []
        for j in range(len(_simulate_list[len(_simulate_list)-1])):
            _industryCO = _simulate_list[len(_simulate_list)-1][j]
            if _industryCO in industryCOs_product:
                # 从每一个优势产业开始向外找，一步联系值超过的节点
                _df_industryCO_proximity = df_product_net.loc[((df_product_net['Source']==_industryCO) | 
                                                               (df_product_net['Target']==_industryCO))&
                                                              (df_product_net['Weight']>0.55)]
                if len(_df_industryCO_proximity)>0:
                    _industryCOs_1 = _df_industryCO_proximity.loc[:,'Source'].unique()
                    _industryCOs_2 = _df_industryCO_proximity.loc[:,'Target'].unique()

                    for m in range(len(_industryCOs_1)):
                        _industryCO_ = _industryCOs_1[m]
                        if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                            _possible_list.append(_industryCO_)
                            ls_a=[]
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_forcast.append(ls_a)
                            #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                            #f.write(ss)
                            c= c+1

                    for m in range(len(_industryCOs_2)):
                        _industryCO_ = _industryCOs_2[m]
                        if  (_industryCO_ not in _exist_list) and (_industryCO_!=_industryCO) :
                            _possible_list.append(_industryCO_)
                            ls_a=[]
                            ls_a.append(t)
                            ls_a.append(_industryCO)
                            ls_a.append(_industryCO_)
                            ls_forcast.append(ls_a)
                            #ss =str(t) + ',' + _industryCO + ','+ _industryCO_ + '\n'
                            #f.write(ss)
                            c= c+1
        if c == 0:
            flag=0
        else:
            flag = 1
            t = t + 1
            _possible_list = list(set(_possible_list))
            _simulate_list.append(_possible_list)
    return ls_forcast